To evaluate the merits and downfalls of Decision Trees, we'll be looking at a dataset concerning the outcome of colicking horses. Each row details a horse experiencing colic, which is a general term referring to when a horse appears to be suffering from abdominal pain. Each horse has variables associated with it, such as its `age`, `pulse`, and `respiratory_rate`, and each horse also has a reported `outcome`: Whether the horse `lived`, `died`, or was `euthanized` as a result of its colic.

Before we proceed further, we'll take a look at all of the data points tracked for each horse.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


horse_df = pd.read_csv("/kaggle/input/horse-colic/horse.csv")

# Print all of the column names
print(horse_df.columns.tolist())

# Print the number of rows and columns
print("Rows: {0}, Columns: {1}".format(len(horse_df), len(horse_df.columns.tolist())))

# Print the first five entries
print(horse_df.head())

# Print how many rows have one or more null entries
print(horse_df.isnull().any(axis=1).value_counts())

# Print the number of null entries per column
print(horse_df.isnull().sum().div(len(horse_df)))

As we can see, there are 28 data points for each horse, although some, such as `hospital_number`, aren't useful for training a classification algorithm. Also, with only 299 entries, it may be difficult to train accurate models that are reliant on large datasets. For this reason, I wouldn't expect good results from a neural network.
This particular dataset also seems to be sparse. The columns `abdomo_apperance` and `abdomo_protein` only have values for two of the first five rows, and `peripheral_pulse`, `temp_of_extremities` and `mucous_membrane` are also missing entries for at least one of the first five entries. After running `horse_df.isnull().any(axis=1).value_counts()`, we can see that there are only 6 entries which have no missing values for any of the columns. The column `nasogastric_reflux_ph`, in particular, is `NaN` for over 80% of entries, while 9 columns are missing values for at least 20% of entries.

```
surgery                  0.000000
age                      0.000000
hospital_number          0.000000
rectal_temp              0.200669
pulse                    0.080268
respiratory_rate         0.193980
temp_of_extremities      0.187291
peripheral_pulse         0.230769
mucous_membrane          0.157191
capillary_refill_time    0.107023
pain                     0.183946
peristalsis              0.147157
abdominal_distention     0.187291
nasogastric_tube         0.347826
nasogastric_reflux       0.354515
nasogastric_reflux_ph    0.822742
rectal_exam_feces        0.341137
abdomen                  0.394649
packed_cell_volume       0.096990
total_protein            0.110368
abdomo_appearance        0.551839
abdomo_protein           0.662207
outcome                  0.000000
surgical_lesion          0.000000
lesion_1                 0.000000
lesion_2                 0.000000
lesion_3                 0.000000
cp_data                  0.000000
```

From the above data, there are only 9 columns which contain no missing values, two of which - `hospital_number` and `cp_data` - aren't helpful for classification, as `hospital_number` is __essentially_ a UUID for the case and `cp_data` has the following description in the documentation: `"this variable is of no significance since pathology data is not included or collected for these cases."` 


One of the remaining 7, `outcome`, is the variable that the classifiers will be trying to predict, so there are only 6 useful columns without missing values that can be used as input for out classification task. (Later in this document we will make use of additional columns by apply techniques to deal with missing entries).


It's also important to check the dataset for any class imbalance. Ideally, the dataset will have an even split of each class, but realistically that likely won't be the case.





In [ ]:
print(horse_df["outcome"].value_counts().divide(len(horse_df)))

Around 60% of horses in the dataset survived, while around 25% died and less than 15% were euthanized. Naively classifying all inputs as having `lived` will result in a `59%` success rate, so using accuracy as the sole validation metric may be misleading. Additional metrics, such as precision and recall, will also be considered. Some basic functions to calculate these metrics are provided below.


We'll also split our data into a test set and training set.

In [ ]:
from sklearn.model_selection import train_test_split


def get_accuracy(confusion_matrix):
    return sum([confusion_matrix[i][i] for i in range(len(confusion_matrix))]) / np.sum(confusion_matrix)


def get_recalls(confusion_matrix):
    return [confusion_matrix[i][i]/np.sum(confusion_matrix, axis=1)[i] for i in range(len(confusion_matrix))]
    

def get_precisions(confusion_matrix):
    return [confusion_matrix[i][i]/np.sum(confusion_matrix, axis=0)[i] for i in range(len(confusion_matrix))]


def get_weighted_f1(confusion_matrix):
    class_freq = np.sum(confusion_matrix, axis=0)
    recalls, precisions = get_recalls(confusion_matrix), get_precisions(confusion_matrix)
    return sum([(2 * frequency * precision * recall) / (precision + recall) for frequency, precision, recall in zip(class_freq, precisions, recalls)]) / sum(class_freq)


# These variables have a dtype of int64, but are nominal classes, so they'll just be converted to strings for convenience later on.
for cat in ["lesion_1", "lesion_2", "lesion_3"]:
    horse_df[cat] = horse_df[cat].astype(str)
horse_train, horse_test = train_test_split(horse_df, test_size=0.25, random_state=100)

Now, with most of the setup out of the way, we'll take a look at training a simple decision tree to predict the outcome of a given horse.


Decision trees work by making a series of classification decisions to maximize information gain (`IG()`), where information is gained when a certain classification decision results in a reduction in the entropy of the dataset. Entropy (`E()`) is a metric which evaluates class uniformity (or data impurity) - `E() == 0` indicates that all of the data belongs to a single class, while `E() == 1` denotes that the data contains multiple classes, with each class having the same representation. 


To see how this works, consider there is a dataset with two classes `True` and `False`, with 20 occurences of each class. Thus, `E() == 1`. The goal of our decision tree would be to split the dataset into two subsets, each with `E() == 0`, as one dataset will contain all `True` occurences and the other will contain all `False` occurences.


Below, we'll take a look at both the entropy and potential information gain in the horse colic dataset.

In [ ]:
import math

# This specific dataset has three classes, but subproblems of this dataset may only look at some of those classes e.g. lived or died
def get_entropy(value_counts, class_count=3):
    entropy = 0
    row_count = sum(value_counts)
    for val in value_counts:
        prob_class = val/row_count
        entropy -= 0 if prob_class == 0 else prob_class * math.log(prob_class, class_count)
    return entropy


def get_information_gain(value_counts_before, value_counts_after_arr):
    if any([el.empty for el in value_counts_after_arr]):
        return 0
    row_count = sum(value_counts_before)
    entropy_before = get_entropy(value_counts_before, row_count)
    entropy_after = sum([get_entropy(value_counts, row_count)*(sum(value_counts)/row_count) for value_counts in value_counts_after_arr])
    return entropy_before - entropy_after


def split_dataset_categorical(df, column, output_class):
    return [df.loc[df[column] == cutoff][output_class].value_counts() for cutoff in df[column].unique().tolist()]

print("Entropy: {0}".format(get_entropy(horse_train["outcome"].value_counts())))


print("Information Gain when spliting on age (young/adult): {0}".format(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                             split_dataset_categorical(horse_train, "age", "outcome"))))
print("Information Gain when spliting on surgery (yes/no): {0}".format(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                            split_dataset_categorical(horse_train, "surgery", "outcome"))))





In the subset of data we are working with, we see that `E() ~= 0.87`. Also, we see that `IG(outcome, age) ~= 0.02` and `IG(outcome, surgery) ~= 0.02`. But what does this mean?
 
 
As discussed above, a value of `E() == 1` denotes the highest level of data impurity, while `E() == 0` denotes pure data - all of the data belongs to a single class. Thus, with `E() ~= 0.82`, we can infer that the dataset is slightly imbalanced. 
 
 
Information gain, meanwhile, is a metric to measure how much a classificaiton decision separates an impure dataset into pure subsets. `IG() > 0` signifies that the classification decision separates the data along class boundaries *to some extent*, with larger values indicating a more accurate split between classes. The formula is given below.

`IG(Y, X) == E(Y) - E(Y | X)`

If `IG(Y, X) == E(Y)`, then we can see that the classification decision separated all `K` classes into `K` subsets, where every instance of class `Ki` belongs to subset `Ki`. Meanwhile, as `IG(Y, X) --approaches--> 0`, the classification decision failed to separate the classes among subsets any better than they were previously.


Thus, `age` and `surgery` are both individually unsuccessful at helping to predict the outcome of a colicking horse. Are other columns any better?

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt



def build_heatmap(t_df, output_column, filter_column):
    df = t_df.fillna("NaN")
    rows = df[filter_column].unique().tolist()
    cols = df[output_column].unique().tolist()
    val_counts = df[output_column].value_counts().to_dict()
    heatmap = []
    for category in rows:
        filtered_subset = [0, 0, 0]
        value_count_dict = df.loc[df[filter_column] == category][output_column].value_counts().to_dict()
        for output_category in value_count_dict:
            filtered_subset[cols.index(output_category)] = value_count_dict[output_category] / val_counts[output_category]
        heatmap.append(filtered_subset)
    return pd.DataFrame(heatmap, index=rows, columns=cols)


f, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
sn.set(font_scale=2)
sn.heatmap(build_heatmap(horse_train, "outcome", "surgery"), annot=True, cmap="YlGnBu", ax=axes[0])
sn.heatmap(build_heatmap(horse_train, "outcome", "age"), annot=True, cmap="YlGnBu", ax=axes[1])
sn.heatmap(build_heatmap(horse_train, "outcome", "surgical_lesion"), annot=True, cmap="YlGnBu", ax=axes[2])

axes[0].set_title('Surgery: IG() = {0}'.format(round(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                             split_dataset_categorical(horse_train, "surgery", "outcome")), 3)))
axes[1].set_title('Age: IG() = {0}'.format(round(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                             split_dataset_categorical(horse_train, "age", "outcome")), 3)))
axes[2].set_title('Surgical Lesion: IG() = {0}'.format(round(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                             split_dataset_categorical(horse_train, "surgical_lesion", "outcome")), 3)))

print(horse_train["outcome"].value_counts())

From the above, we see that `IG(outcome, age) ~= 0.02`, `IG(outcome, surgery) ~= 0.02`, and `IG(outcome, surgical_lesion) ~= 0.06`.

Using heatmaps, it's also possible to see how each criteria splits the data. `surgical_lesion`, for example, is useful for predicting whether a horse died or not - 89% of all horses that had surgical lesions died, while only 11% of horses that did not have surgical lesions died. However, nearly the same number of horses that had surgical lesions lived (51%) as those that did not have surgical lesions (49%). The `age` heatmap suggests that while the vast majority of colicking horses in the dataset were adults, those that were young almost always died.



At this point, we can see that there are several plausible methods by which the dataset can be split into subsets. We also know that out of the options that have been evaluated currently, we can split the data using `surgical_lesion [yes\no]`, but with this single decision a classifier will still always classify any input as `lived`, because `lived` makes up the majority of occurences for both datasets. The only split of the three above that would yield different behavior would be `age [young/adult]` - adults would be predicted to have `lived`, while young horses would be predicted to have `died`, as there more young horses died than lived (`(0.16 * 63 ~= 10) > (0.055 * 128 ~= 7)`). 


A decision tree, of course, is made up of many of these splits. While the root of the decision tree will be the node that leads to the most information gain, each subsequent branch will recursively continue this pattern, until the tree has reached a maximum height. This maximum height hyperparameter **hopefully** prevents overfitting, with the goal of preventing the tree from fitting to noise. If `hospital_number` was included as input during training, for instance, the tree might learn to make predictions based on the hospital number of the horse if it leads to better classification, even though such a correlation almost certianlty does not exist in practice (technically, vets could be making decisions based on these hospital numbers, and a model might appropriately detect this - but this chance is infinitesimaly small).

With this understanding of a decision tree, a simple one will be constructed below.


In [ ]:
from collections import deque
from sklearn.metrics import confusion_matrix

class Node():
    def __init__(self, df, level, column=None, split_value=None, ig=0):
        self.member = None
        self.non_member = None
        self.conditinal = None
        self.column = column
        self.split_value = split_value
        self.ig = ig
        self.df = df
        self.level = level
    
    def eval_conditional(self, lval, rval):
        if not self.conditional:
            return False
        elif "==" in self.conditional:
            return lval == rval
        elif ">=" in self.conditional:
            return lval >= rval
        else:
            return lval < rval
        
    def __str__(self):
        return "Node(Column: {0}, Split Value: {1}, IG: {2}, Level: {3})".format(self.column, self.split_value, self.ig, self.level)


class DecisionTree():
    def __init__(self, df, prediction_column, height):
        self.height = height
        self.root = Node(df.copy(), 0)
        self.prediction_column = prediction_column
        self.input_columns = df.columns.tolist()
        self.input_columns.remove(prediction_column)
    
    def train(self):
        q = deque([self.root])
        while q:
            curr_node = q.pop()
            if curr_node.level == self.height + 1:
                break
            self.branch(curr_node)
            if curr_node.member:
                q.extendleft([curr_node.member, curr_node.non_member])
        return self.root
             
    
    def branch(self, node):
        df = node.df
        input_columns = df.columns.tolist()
        input_columns.remove(self.prediction_column)
        for column in input_columns:
            for unique_value in horse_train[column].unique():
                subsets = [df.loc[df[column] == unique_value], df.loc[df[column] != unique_value]]
                ig = get_information_gain(df[self.prediction_column].value_counts(), [el[self.prediction_column].value_counts() for el in subsets])
                if node.ig < ig or (node.ig == ig and node.ig > 0):
                    node.column = column
                    node.split_value = unique_value
                    node.ig = ig
                    node.conditional = "=="
                    node.member = Node(subsets[0].copy(), node.level + 1)
                    node.non_member = Node(subsets[1].copy(), node.level + 1)
                if horse_train[column].dtype in ["int64", "float64"]:
                    subsets = [df.loc[df[column] >= unique_value], df.loc[df[column] < unique_value]]
                    ig = get_information_gain(df[self.prediction_column].value_counts(), [el[self.prediction_column].value_counts() for el in subsets])
                    if node.ig < ig:
                        node.column = column
                        node.split_value = unique_value
                        node.ig = ig
                        node.conditional = ">="
                        node.member = Node(subsets[0].copy(), node.level + 1)
                        node.non_member = Node(subsets[1].copy(), node.level + 1)
                    subsets = [df.loc[df[column] < unique_value], df.loc[df[column] >= unique_value]]
                    ig = get_information_gain(df[self.prediction_column].value_counts(), [el[self.prediction_column].value_counts() for el in subsets])
                    if node.ig < ig:
                        node.column = column
                        node.split_value = unique_value
                        node.ig = ig
                        node.conditional = "<"
                        node.member = Node(subsets[0].copy(), node.level + 1)
                        node.non_member = Node(subsets[1].copy(), node.level + 1)
                    
    def predict(self, data):
        curr_node = self.root
        while curr_node.member:
            curr_node = curr_node.member if curr_node.eval_conditional(data[curr_node.column], curr_node.split_value) else curr_node.non_member
        return ("The horse outcome for ({0}) is: {1}".format(str(data), curr_node.df[self.prediction_column].value_counts().idxmax()), curr_node.df[self.prediction_column].value_counts().idxmax())
    
    def predict_df(self, tdf, labels):
        df = tdf.copy()
        df["prediction"] = df.apply(lambda row: self.predict(row.to_dict())[1], axis=1)
        return confusion_matrix(df[self.prediction_column], df["prediction"], labels=labels)
    
    def naive_prediction(self, tdf, labels):
        df = tdf.copy()
        max_freq_category = df[self.prediction_column].value_counts().idxmax()
        df["prediction"] = df.apply(lambda row: max_freq_category, axis=1)
        return confusion_matrix(df[self.prediction_column], df["prediction"], labels=labels)
    
    def display(self):
        q = deque([([], self.root)])
        subsets = []
        while q:
            criteria, node = q.pop()
            if node.level <= self.height:
                print("Criteria: {0}, Node: {1}, Split: {2}".format(str(criteria), node, node.df[self.prediction_column].value_counts().to_dict()))
                if node.member:
                    if node.conditional == "==":
                        q.extendleft([(criteria + [str(node.column) + " == " + str(node.split_value)], node.member), (criteria + [str(node.column) + " != " + str(node.split_value)], node.non_member)])
                    elif node.conditional == ">=":
                        q.extendleft([(criteria + [str(node.column) + " >= " + str(node.split_value)], node.member), (criteria + [str(node.column) + " < " + str(node.split_value)], node.non_member)])
                    else:
                        q.extendleft([(criteria + [str(node.column) + " < " + str(node.split_value)], node.member), (criteria + [str(node.column) + " >= " + str(node.split_value)], node.non_member)])
                else:
                    subsets.append((str(criteria), node.df[self.prediction_column].value_counts().to_dict(), len(node.df)/len(self.root.df)))
            else:
                subsets.append((str(criteria), node.df[self.prediction_column].value_counts().to_dict(), len(node.df)/len(self.root.df)))
        print("Data subsets:")
        for subset in subsets: 
            print(subset)
            
            
horse_train_stripped = horse_train.drop(columns=[el for el in horse_train.columns.to_list() if el not in ["surgery", "age", "outcome", "surgical_lesion"]])     
model = DecisionTree(horse_train_stripped, "outcome", 3)
model.train()
print(model.predict({"surgery": "no", "age": "adult", "surgical_lesion": "yes"}))

At this point, we finally have a working decision tree, which is a `DecisionTree` made up of `Node`s, where each `Node` represents a decision being made. Each node also has two children - a `Node` for the horses that met its decision criteria, `node.member`, and those that did not meet the criteria, in `node.non_member`. For example, if a `Node` has `node.column == "surgery"` and `node.split_value == "yes"`, then it will have a `Node member_child == node.member` that has all of the horses in the `node` dataset that had surgery, and a `Node non_member_cihld == node.non_member` that has all of the horses in the previous dataset that did not have surgery. Thus, these nodes will continue to filter the data into subsets consisting of more homogenous data until either no more decisions can be made, or the depth of the tree has been reached.


A `DecisionTree` with `height == 3` is trained on the `horse_train` dataset, with all columns besides `surgery`, `age`, `outcome`, and `surgical_lesion` dropped. Possible splits are evaluated and splits that lead to the most information gain are made, and then this process continues recursively with the child nodes. Then, after the `DecisionTree` has been trained it is given a horse to make a prediction for, with `{"surgery": "no", "age": "adult", "surgical_lesion": "yes"}`. We can see that the predicted outcome for such a horse is death, which matches possible intuition - if a horse had an operable lesion but didn't recieve surgery, it makes sense that such a horse would die.


To see how the trained model performs on all of the training set, we'll use the `predict_df()` function, and we'll evaluate the metrics of the model using the metric functions defined near the top of the notebook. The model will be juxtaposed with a naive model that always predicts a horse `lived`, regardless of the input.

In [ ]:
conf_matrix = model.predict_df(horse_train_stripped, ["lived", "died", "euthanized"])
conf_matrix_naive = model.naive_prediction(horse_train_stripped, ["lived", "died", "euthanized"])
print("Naive Recall: {0}, Model Recall: {1}".format(get_recalls(conf_matrix_naive), get_recalls(conf_matrix)))
print("Naive Precision: {0}, Model Precision: {1}".format(get_precisions(conf_matrix_naive), get_precisions(conf_matrix)))
print("Naive Accuracy: {0}, Model Accuracy: {1}".format(get_accuracy(conf_matrix_naive), get_accuracy(conf_matrix)))
print("Naive Weighted F1: {0}, Model Weighted F1: {1}".format(get_weighted_f1(conf_matrix_naive), get_weighted_f1(conf_matrix)))

The trained model has comparable precisions across all classes, but has low recall for `died` and `euthanized` with high recall for `lived`. The comparably low recall for `died` and `euthanized` can be explained by the class imbalance in the dataset, as the model is still biased towards classifying horeses as having `lived`, causing a high false negative rate for the other two classes. The model does achieve higher accuracy than the naive model, with an accuracy of `0.62` versus `0.57`, and the model has a weighted F1 score of `0.65`, whereas the division-by-zero that happens during the calculation of the naive model's precision results in `NaN` for both the precision and weighted F1 scores.


These metrics arefor when the model is evaluated on the training dataset, however, and as a result are inevitably misleading. A model can achieve 100% accuracy for the training set and still achieve only 0% in the test set if it overfits to noise in the data. While this is unlikely to be the case with this basic `DecisionTree`, accurate metrics should still be evaluated.

In [ ]:
horse_test_stripped = horse_test.drop(columns=[el for el in horse_test.columns.to_list() if el not in ["surgery", "age", "outcome", "surgical_lesion"]])    
conf_matrix = model.predict_df(horse_test_stripped, ["lived", "died", "euthanized"])
conf_matrix_naive = model.naive_prediction(horse_test_stripped, ["lived", "died", "euthanized"])
print("Naive Recall: {0}, Model Recall: {1}".format(get_recalls(conf_matrix_naive), get_recalls(conf_matrix)))
print("Naive Precision: {0}, Model Precision: {1}".format(get_precisions(conf_matrix_naive), get_precisions(conf_matrix)))
print("Naive Accuracy: {0}, Model Accuracy: {1}".format(get_accuracy(conf_matrix_naive), get_accuracy(conf_matrix)))
print("Naive Weighted F1: {0}, Model Weighted F1: {1}".format(get_weighted_f1(conf_matrix_naive), get_weighted_f1(conf_matrix)))

For the testing set, class recalls and precisions for `euthanized` and `died` drop, while the accuracy and weighted F1 remain similar. The `DecisionTree` model now achieves worse accuracy than the naive model. Overall, the performance is mediocre at best.


So how can the model be improved?


First, the other informative column with no missing values, `lesion_1` will be incorporated.

In [ ]:
def build_and_eval_dt(df_train, df_test, height, cols, prediction_column, conf_matrix_cols):
    cols = cols.copy()
    cols.append(prediction_column)
    df_train_stripped = df_train.drop(columns=[el for el in df_train.columns.to_list() if el not in cols])
    df_test_stripped = df_test.drop(columns=[el for el in df_test.columns.to_list() if el not in cols])
    model = DecisionTree(df_train_stripped, prediction_column, height)
    model.train()
    train_conf_matrix = model.predict_df(df_train_stripped, conf_matrix_cols)
    test_conf_matrix = model.predict_df(df_test_stripped, conf_matrix_cols)
    print("Training Recall: {0}, Testing Recall: {1}".format(get_recalls(train_conf_matrix), get_recalls(test_conf_matrix)))
    print("Training Precision: {0}, Testing Precision: {1}".format(get_precisions(train_conf_matrix), get_precisions(test_conf_matrix)))
    print("Training Accuracy: {0}, Testing Accuracy: {1}".format(get_accuracy(train_conf_matrix), get_accuracy(test_conf_matrix)))
    print("Training Weighted F1: {0}, Testing Weighted F1: {1}".format(get_weighted_f1(train_conf_matrix), get_weighted_f1(test_conf_matrix)))


build_and_eval_dt(horse_train, horse_test, 4, ["surgery", "age", "surgical_lesion", "lesion_1"], "outcome", ["lived", "died", "euthanized"])

The accuracy of the model increased, but precision and recall of the undersampled classes `died` and `euthanized` decreased, so much so that `euthanized` had a recall of `0.12` for the training set and `0.0` for the testing set. But why? 

In [ ]:
model.display()

After including `lesion_1`, the `DecisionTree` splits the data first based on `lesion_1 == 0`, which denotes whether or not the horse had an operable lesion. Intuitively, such a criterion is useful. The outcomes of horses with operable lesions should be statistically different than those without operable lesions. The first `Node` of the next level also seems logical, as young horses should have different outcomes than old horses. The next `Node`, meanwhile, seems peculiar. According to the documentation, a `lesion_1` of `3111` represents a horse that had a "simple mechanical obturation of the large colon". The next level has a similar node, `lesion_1 == 2205`, where `2205` means the horse had a "paralytic strangulation/torsion of the small intestine". Other lesions that the model splits on are `3205` and `2124`.

While it's plausible that these specific lesions are powerful predictors for the outcome of a colicking horse, the `DecisionTree` may be overfitting to noise.

In [ ]:
horse_train_stripped = horse_train.drop(columns=[el for el in horse_train.columns.to_list() if el not in ["surgery", "age", "outcome", "surgical_lesion", "lesion_1"]])     
horse_test_stripped = horse_test.drop(columns=[el for el in horse_test.columns.to_list() if el not in ["surgery", "age", "outcome", "surgical_lesion", "lesion_1"]])  
print("Lesion 3111 - Training vs Test")
print(horse_train_stripped.loc[horse_train_stripped["lesion_1"] == "3111"]["outcome"].value_counts())
print(horse_test_stripped.loc[horse_test_stripped["lesion_1"] == "3111"]["outcome"].value_counts())
print("Lesion 2205 - Training vs Test")
print(horse_train_stripped.loc[horse_train_stripped["lesion_1"] == "2205"]["outcome"].value_counts())
print(horse_test_stripped.loc[horse_test_stripped["lesion_1"] == "2205"]["outcome"].value_counts())

Looking at the training sets, it becomes apparent why the `DecisionTree` split based on lesions `3111` and `2205`. 92% of lesion `3111` cases lived in the training set, and 100% of lesion `2205` cases died. For lesion `3111`, the trend continues into the test set, with 100% of cases living. But for lesion `2205`, only 14% of cases in the test set died. At least for lesion `2205`, the model fit to noise in the training set.


There's no more dimensions to add to the model that have no missing values, so without addressing the `NaN` values, the `DecisionTree` model is able to obtain the following results:


```
Naive Model - One Decision Node
Testing Recall: [1.0, 0.0, 0.0]
Testing Precision: [0.6666666666666666, nan, nan]
Testing Accuracy: 0.6666666666666666
Testing Weighted F1: nan

Basic Model - Three dimensions, no pre-processing, no NaNs
Testing Recall: [0.86, 0.14285714285714285, 0.18181818181818182]
Testing Precision: [0.7166666666666667, 0.2, 0.4]
Testing Accuracy: 0.6266666666666667
Testing Weighted F1: 0.6643434343434342

Basic Model - Four dimensions, no pre-processing, no NaNs
Testing Recall: [0.92, 0.14285714285714285, 0.0]
Testing Precision: [0.71875, 0.18181818181818182, nan]
Testing Accuracy: 0.64
Testing Weighted F1: nan
```


To improve the `DecisionTree`'s performance further, it's imperative that at least some of the `15+` informative columns with missing values are provided to the model. But what happens if some of the additional columns are provided without any pre-processing?


First, let's take a look at one of the additional columns, `pain`, to get an idea of the potential information gain.

In [ ]:
print("Information Gain when spliting on pain level (severe_pain/depressed/extreme_pain/mild_pain/alert): {0}".format(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                            split_dataset_categorical(horse_train, "pain", "outcome"))))
print("Information Gain when spliting on age (young/adult): {0}".format(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                            split_dataset_categorical(horse_train, "age", "outcome"))))
print("Information Gain when spliting on surgery (yes/no): {0}".format(get_information_gain(horse_train["outcome"].value_counts(), 
                                                                                            split_dataset_categorical(horse_train, "surgery", "outcome"))))
axis = plt.axes()
sn.heatmap(build_heatmap(horse_train, "outcome", "pain"), annot=True, cmap="YlGnBu", ax=axis)
axis.set_title('Outcomes vs Pain Level')

The potential information gain from `pain` is significantly larger than that of both `age` and `surgery`, so hopefully the model's performance will be able to improve with this added dimension.

In [ ]:
build_and_eval_dt(horse_train, horse_test, 4, ["surgery", "age", "surgical_lesion", "lesion_1", "pain"], "outcome", ["lived", "died", "euthanized"])

The recall for `died` and the precision for `lived` increased significantly compared to past models, but the accuracy decreased slightly and the model still failed to predict any `euthanized` cases properly.

We'll add a few more dimensions to see how performance is affected.

In [ ]:
build_and_eval_dt(horse_train, horse_test, 6, ["surgery", "age", "outcome", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen"], "outcome", ["lived", "died", "euthanized"])

With the additional dimensions, the `DecisionTree` is able to correctly classify at least some occurences of `euthanized` in the testing set, although only 18%. The accuracy and weighted F1 are marginally lower than they were with previous models.

As an aside, while this decision tree implementation sets a maximum depth, some decision tree implementations support pruning, which involves removing certain nodes from the tree to improve performance. While an actual implementation of pruning is out of scope for this notebook, just know that there are several techniques for pruning, but the goal of all of them is to reduce overfitting by reducing model complexity. 

Finally, we will incorporate techniques to handle the missing values. First, let's check how many rows have missing values with only this subset of rows. (From the beginning of this notebook, 293/299 had missing values when all columns were included).

In [ ]:
# Print how many rows have one or more null entries
horse_train_stripped = horse_train.drop(columns=[el for el in horse_train.columns.to_list() if el not in ["surgery", "age", "outcome", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen"]]) 
print(horse_train_stripped.isnull().any(axis=1).value_counts())

133/224, or 59% of of the entries have data missing across at least one dimension. This demerits one of the more popular and arguably the easiest and most intuitive approach to addressing missing values - listwise deletion - as 59% of the dataset would be lost if all entries with a missing value were omitted. Listwise deletion is already ill-advised for datasets with far less data missing, as any removal of entries detracts from potential learning and potentially increases the bias of the model, so in this particular case it will be especially detrimental to use. 

Instead of deletion, missing values can be addressed by utilizing imputation. Missing values can be infered from the other, available values for that variable. For example, if there is a variable that is missing a value for only one entry, and for 75% of the other entries the value of the variable is `X`, with imputation the missing value can be replaced with `X`. There are flaws with this approach - it's possible that the missing value was not `X`, causing the model to train on fraudulent data - but imputation is nevertheless useful, so we will incorporate it into the model.

Average imputation will be used, where the average value of continuous variables and the mode of discrete variables will replace the missing values for each variable.

In [ ]:
cols = ["surgery", "age", "outcome", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen"]

horse_train_stripped = horse_train.drop(columns=[el for el in horse_train.columns.to_list() if el not in cols])     
horse_test_stripped = horse_test.drop(columns=[el for el in horse_test.columns.to_list() if el not in cols])  
for category in cols:
    horse_train_stripped[category].fillna(horse_train_stripped[category].mode()[0], inplace=True)
    horse_test_stripped[category].fillna(horse_train_stripped[category].mode()[0], inplace=True)
model = DecisionTree(horse_train_stripped, "outcome", 6)
model.train()
train_conf_matrix = model.predict_df(horse_train_stripped, ["lived", "died", "euthanized"])
test_conf_matrix = model.predict_df(horse_test_stripped, ["lived", "died", "euthanized"])
print("Training Recall: {0}, Testing Recall: {1}".format(get_recalls(train_conf_matrix), get_recalls(test_conf_matrix)))
print("Training Precision: {0}, Testing Precision: {1}".format(get_precisions(train_conf_matrix), get_precisions(test_conf_matrix)))
print("Training Accuracy: {0}, Testing Accuracy: {1}".format(get_accuracy(train_conf_matrix), get_accuracy(test_conf_matrix)))
print("Training Weighted F1: {0}, Testing Weighted F1: {1}".format(get_weighted_f1(train_conf_matrix), get_weighted_f1(test_conf_matrix)))

Accuracy increased slightly from the previous model after imputation for the training set (77% to 79%), but decreased greatly from the previous model for the testing set (64% to 57%).  Overall, it doesn't seem average imputation helped much, but this could be because all missing values were replaced with the mode for that variable. Performance might improve if continuous datatypes, like `int64` and `float64`, were replaced with their means instead.

In [ ]:
def build_and_eval_dt_imputation(df_train, df_test, height, columns, prediction_column, conf_matrix_cols):
    columns = columns.copy()
    columns.append(prediction_column)
    df_train_stripped = df_train.drop(columns=[el for el in df_train.columns.to_list() if el not in columns])
    df_test_stripped = df_test.drop(columns=[el for el in df_test.columns.to_list() if el not in columns])
    for category in columns:
        if df_train[category].dtype in ["int64", "float64"]:
            df_train_stripped[category].fillna(df_train_stripped[category].mean(), inplace=True)
            df_test_stripped[category].fillna(df_train_stripped[category].mean(), inplace=True)
        else:
            df_train_stripped[category].fillna(df_train_stripped[category].mode()[0], inplace=True)
            df_test_stripped[category].fillna(df_train_stripped[category].mode()[0], inplace=True)
    model = DecisionTree(df_train_stripped, prediction_column, height)
    model.train()
    train_conf_matrix = model.predict_df(df_train_stripped, conf_matrix_cols)
    test_conf_matrix = model.predict_df(df_test_stripped, conf_matrix_cols)
    print("Training Recall: {0}, Testing Recall: {1}".format(get_recalls(train_conf_matrix), get_recalls(test_conf_matrix)))
    print("Training Precision: {0}, Testing Precision: {1}".format(get_precisions(train_conf_matrix), get_precisions(test_conf_matrix)))
    print("Training Accuracy: {0}, Testing Accuracy: {1}".format(get_accuracy(train_conf_matrix), get_accuracy(test_conf_matrix)))
    print("Training Weighted F1: {0}, Testing Weighted F1: {1}".format(get_weighted_f1(train_conf_matrix), get_weighted_f1(test_conf_matrix)))
    

build_and_eval_dt_imputation(horse_train, horse_test, 6, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen"], "outcome", ["lived", "died", "euthanized"])

Using the mean to perform imputation for continuous datatypes seems to have worsened model performance, with testing accuracy further dropping from `64% -> 57% -> 49%`, while training performance continued to improve. With imputation, the model seems to overfit to noise introduced by imputation. This may be a result of the large number of missing entries being replaced with their respective mode/mean.

Does adding more columns improve the performance? And how does the model compare with versus without imputation?

In [ ]:
print("With Imputation:")
build_and_eval_dt_imputation(horse_train, horse_test, 6, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen", "abdominal_distention", "packed_cell_volume", "total_protein", "rectal_temp", "capillary_refill_time"], "outcome", ["lived", "died", "euthanized"])
print("Without Imputation:")
build_and_eval_dt(horse_train, horse_test, 6, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen", "abdominal_distention", "packed_cell_volume", "total_protein", "rectal_temp", "capillary_refill_time"], "outcome", ["lived", "died", "euthanized"])

After adding additional variables to the model, and after evaluating the model with and without imputation, the following results are obtained.

```
Naive Model - One Decision Node
Testing Recall: [1.0, 0.0, 0.0]
Testing Precision: [0.6666666666666666, nan, nan]
Testing Accuracy: 0.6666666666666666
Testing Weighted F1: nan

Basic Model - Three dimensions, no pre-processing, no NaNs
Testing Recall: [0.86, 0.14285714285714285, 0.18181818181818182]
Testing Precision: [0.7166666666666667, 0.2, 0.4]
Testing Accuracy: 0.6266666666666667
Testing Weighted F1: 0.6643434343434342

Basic Model - Four dimensions, no pre-processing, no NaNs
Testing Recall: [0.92, 0.14285714285714285, 0.0]
Testing Precision: [0.71875, 0.18181818181818182, nan]
Testing Accuracy: 0.64
Testing Weighted F1: nan

With Average Imputation - 15 Dimensions:
Testing Recall: [0.72, 0.5714285714285714, 0.2727272727272727]
Testing Precision: [0.7659574468085106, 0.32, 1.0]
Testing Accuracy: 0.6266666666666667
Testing Weighted F1: 0.6190496330702515

Without Average Imputation - 15 Dimensions:
Testing Recall: [0.66, 0.42857142857142855, 0.45454545454545453]
Testing Precision: [0.7674418604651163, 0.35294117647058826, 0.3333333333333333]
Testing Accuracy: 0.5866666666666667
Testing Weighted F1: 0.5715467328370554
```

The best accuracy is the basic model with only four dimensions provided, although it fails to classify any `euthanized` horses properly and thus has a `NaN` weighted F1 score. The next best model is when there are only three basic dimensions provided - `age`, `surgery`, and `surgical_lesion` - with no pre-processing, and this is also the model with the greatest weighted F1. The difference between performance amongst the models is negligible, with the main distinction between models being whether they correctly classify any `euthanized` horses. 

Still, the model with the best accuracy is the naive model which always predicts that a given horse `lived`. Additional dimensions seem to be leading to overfitting and therefore worsening model testing performance, but will adding five _more_ dimensions improve the performance of the model at all?

In [ ]:
build_and_eval_dt_imputation(horse_train, horse_test, 8, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen", "abdominal_distention", "packed_cell_volume", "total_protein", "rectal_temp", "capillary_refill_time", "rectal_exam_feces", "nasogastric_tube", "nasogastric_reflux", "abdomo_appearance", "temp_of_extremities"], "outcome", ["lived", "died", "euthanized"])

Surprisingly, with the addition of the dimensions `["rectal_exam_feces", "nasogastric_tube", "nasogastric_reflux", "abdomo_appearance", "temp_of_extremities"]` the `DecisionTree` model is finally able to surpass the performance of the naive model, with a testing accuracy of `69%` and a weighted F1 of `70%`. The model also nears perfect accuracy for the training set, properly classifying `96%` of horses.


```
Testing Recall: [0.74, 0.8571428571428571, 0.2727272727272727]
Testing Precision: [0.8222222222222222, 0.46153846153846156, 0.75]
Testing Accuracy: 0.6933333333333334
Testing Weighted F1: 0.6967017543859648

```

Out of curiosity, how does model performance change when `euthanized` and `died` are merged into a single class? For example, a model may only be required to predict whether a horse lived or died, regardless of whether the cause of death was euthanasia.

All occurences of `euthanized` will be replaced with `died`, and the model will be trained on this new data.


In [ ]:
horse_train_lived_died = horse_train.copy()
horse_test_lived_died = horse_test.copy()
horse_train_lived_died["outcome"].replace("euthanized", "died", inplace=True)
horse_test_lived_died["outcome"].replace("euthanized", "died", inplace=True)

build_and_eval_dt_imputation(horse_train_lived_died, horse_test_lived_died, 8, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen", "abdominal_distention", "packed_cell_volume", "total_protein", "rectal_temp", "capillary_refill_time", "rectal_exam_feces", "nasogastric_tube", "nasogastric_reflux", "abdomo_appearance", "temp_of_extremities"], "outcome", ["lived", "died"])

The accuracy and weighted F1 scores of the model improve from `(69%, 70%)` to `(71%, 71%)`, but this performance gain is miniscule. 

It's possible that the `euthanized` and `died` sets are incongruent and the model is unable to learn to classify horses properly as a result. To test this, all instances of `euthanized` can be removed from the dataset, instead of changing the label of the class. We'll try that below.

In [ ]:
horse_train_lived_died = horse_train.copy()
horse_test_lived_died = horse_test.copy()
horse_train_lived_died = horse_train_lived_died[horse_train_lived_died["outcome"] != "euthanized"]
horse_test_lived_died = horse_test_lived_died[horse_test_lived_died["outcome"] != "euthanized"]

print(horse_train_lived_died["outcome"].value_counts().divide(len(horse_train_lived_died)))
print(horse_test_lived_died["outcome"].value_counts().divide(len(horse_test_lived_died)))

build_and_eval_dt_imputation(horse_train_lived_died, horse_test_lived_died, 8, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen", "abdominal_distention", "packed_cell_volume", "total_protein", "rectal_temp", "capillary_refill_time", "rectal_exam_feces", "nasogastric_tube", "nasogastric_reflux", "abdomo_appearance", "temp_of_extremities"], "outcome", ["lived", "died"])

With all instances of `euthanized` removed, 67% of all horses in the training set `lived` while the remaining 33% `died`. Likewise, 78% of the test set `lived` while 22% of the test set `died`. Thus, the naive model could achieve a testing accuracy of 78%.

The `DecisionTree` greatly outperforms the naive model with 92% accuracy for the training set, but the accuracy of the model drops significantly when given the testing set. The worsened metrics from the training to the testing set suggest overfitting. When a model is given 20 dimensions for a dataset with less than 300 entries, overfitting can be expected, especially when the model being trained is prone to overfitting like a decision tree. Some of the 20 variables are likely uninformative, and such variables will introduce noise and provide no benefit to the model. High dimensionality for a comparatively small dataset also hightens the probability of the test set to have novel combinations of values that were not present in the training set. Combined with the amount of imputation being used for certain variables, the horse colic dataset has several factors that make overfitting likely, and the metrics being observed suggest this is the problem.

Before we proceed any further with dimensonality reduction, let's first test the overfitting hypothesis and see how the model performs with fewer dimensions.

In [ ]:
horse_train_lived_died = horse_train.copy()
horse_test_lived_died = horse_test.copy()
horse_train_lived_died["outcome"].replace("euthanized", "died", inplace=True)
horse_test_lived_died["outcome"].replace("euthanized", "died", inplace=True)

print("Performance when euthanized and died are merged:")
build_and_eval_dt_imputation(horse_train_lived_died, horse_test_lived_died, 5, ["surgery", "age", "surgical_lesion", "lesion_1", "pain"], "outcome", ["lived", "died"])

horse_train_lived_died = horse_train.copy()
horse_test_lived_died = horse_test.copy()
horse_train_lived_died = horse_train_lived_died[horse_train_lived_died["outcome"] != "euthanized"]
horse_test_lived_died = horse_test_lived_died[horse_test_lived_died["outcome"] != "euthanized"]

print("Performance when euthanized cases are removed:")
build_and_eval_dt_imputation(horse_train_lived_died, horse_test_lived_died, 5, ["surgery", "age", "surgical_lesion", "lesion_1", "pain"], "outcome", ["lived", "died"])

```
## With 20 Dimensions, MaxHeight == 8
Performance when euthanized and died are merged:
Training Recall: [0.9609375, 0.8645833333333334], Testing Recall: [0.78, 0.56]
Training Precision: [0.9044117647058824, 0.9431818181818182], Testing Precision: [0.78, 0.56]
Training Accuracy: 0.9196428571428571, Testing Accuracy: 0.7066666666666667
Training Weighted F1: 0.9201722190852626, Testing Weighted F1: 0.7066666666666667

Performance when euthanized cases are removed:
Training Recall: [0.8828125, 1.0], Testing Recall: [0.7, 0.6428571428571429]
Training Precision: [1.0, 0.8076923076923077], Testing Precision: [0.875, 0.375]
Training Accuracy: 0.9214659685863874, Testing Accuracy: 0.6875
Training Weighted F1: 0.9197326316168983, Testing Weighted F1: 0.6637426900584795

-----

## With 5 Dimensions, MaxHeight == 5
Performance when euthanized and died are merged:
Training Recall: [0.84375, 0.7604166666666666], Testing Recall: [0.76, 0.8]
Training Precision: [0.8244274809160306, 0.7849462365591398], Testing Precision: [0.8837209302325582, 0.625]
Training Accuracy: 0.8080357142857143, Testing Accuracy: 0.7733333333333333
Training Weighted F1: 0.8084474780903352, Testing Weighted F1: 0.7679456706281833

Performance when euthanized cases are removed:
Training Recall: [0.8515625, 0.7777777777777778], Testing Recall: [0.78, 0.7857142857142857]
Training Precision: [0.8861788617886179, 0.7205882352941176], Testing Precision: [0.9285714285714286, 0.5]
Training Accuracy: 0.8272251308900523, Testing Accuracy: 0.78125
Training Weighted F1: 0.8256487657937055, Testing Weighted F1: 0.7664553140096618
```

After removing 15 dimensions the performance of the model vastly improved, and the metrics of both the testing and training sets suggest the model is only slightly biased towards the `lived` class. With the only change being the removal of potentially noisy or excessive variables, the testing weighted F1 increased from `71% -> 77%` and `69% -> 78%` for the models when `euthanized` was merged with `died` or removed from the data, respectively. 

With over 20 potentially informative dimensions, it would be naive to select a few variables at random and assume that those lead to the best performance, and it'd be a waste of computational resources and time to try every subset of variables. Therefore, the most informative variables will be determined by measuring each variable's correlation to the `outcome` variable with the Chi Squared test. Principal Component Analysis may also be beneficial, but for now only Chi Squared Test of Independence and One-way Anova will be used.

Chi Squared will be used in the case of categorical variables, while Anova will be used for continuous variables. It should be noted that both tests make assumptions about the data that may not be true in the case of the given data. For example, Anova assumes homoscedasticity, whereas certain variables, such as pulse, likely have heteroscedasticity. Additionally, the Chi Squared test is ideally used on data where the expected values of columns are 5 or more, but this isn't the case for certain variables such as `lesion_1`, as the expected number of `euthanized` horses with a given lesion type would be close to 0. Still, the p-values from the tests will be useful for getting a rough idea of which variables are more predictive of `outcome` than others. 

We'll compute these p-values for the training set, the testing set, and for the combined dataset, just to see if there's any significant deviation between variable distributions between the two subsets.

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
import bisect

# Returns the p-value after performing One-way Anova on `outcome` and `input_col`
def compute_p_anova(df, input_col):
    df_lived = df.loc[df["outcome"] == "lived"][input_col].dropna()
    df_died = df.loc[df["outcome"] == "died"][input_col].dropna()
    df_euthanized = df.loc[df["outcome"] == "euthanized"][input_col].dropna()
    return f_oneway(df_lived, df_died, df_euthanized)[1]

# Returns the p-value after performing the Chi Squared Test of Independence on `outcome` and `input_col`
def compute_p_chi2(df, input_col):
    ct = pd.crosstab(df[input_col], df["outcome"])
    return chi2_contingency(ct)[1]

print("Training set size: {0}".format(len(horse_train)))
print("Testing set size: {0}".format(len(horse_test)))

ordered_vars_train = []
ordered_vars_test = []
ordered_vars_combined = []

for var in horse_df.columns:
    if horse_df[var].dtype not in ["int64", "float64"]:
        train_p, test_p, combined_p = compute_p_chi2(horse_train, var), compute_p_chi2(horse_test, var), compute_p_chi2(horse_df, var)
        print("[{0}] - Training p-value: {1}, Testing p-value: {2}, Combined p-value: {3}".format(var, train_p, test_p, combined_p))
    else:
        train_p, test_p, combined_p = compute_p_anova(horse_train, var), compute_p_anova(horse_test, var), compute_p_anova(horse_df, var)
        print("[{0}] - Training p-value: {1}, Testing p-value: {2}, Combined p-value: {3}".format(var, train_p, test_p, combined_p))
    bisect.insort(ordered_vars_train, (train_p, var))
    bisect.insort(ordered_vars_test, (test_p, var))
    bisect.insort(ordered_vars_combined, (combined_p, var))
print("Variables sorted by p-value: Training set:\n", [el[1] if el[0] < 0.01 else el[1] + "*" for el in ordered_vars_train])
print("Variables sorted by p-value: Testing set:\n", [el[1] if el[0] < 0.01 else el[1] + "*" for el in ordered_vars_test])
print("Variables sorted by p-value: Overall:\n", [el[1] if el[0] < 0.01 else el[1] + "*" for el in ordered_vars_combined])

The `outcome` of a horse is dependent on the majority of variables in the dataset. The only variables that are significantly uncorrelated are `rectal_temp`, `respiratory_rate`, `nasogastic_tube`, `lesion_1`, and `lesion_2`. Several variables reject the null hypothesis of the tests when `p == 0.05`, but fail to reject the null hypothesis when a more strict `p == 0.01` is used: `age`, `hospital_number`, `nasogastric_reflux_ph`, and `cp_data`. As discussed earlier in the notebook, since `hospital_number` is a UUID for each horse (but may technically appear multiple times, if the same horse was treated) and `cp_data` is irrelevant to this dataset, those two variables are irrelevant regardless. 

There are only two variables with `p < 0.01` for the testing set - `packed_cell_volume` and `peripheral_pulse` (`outcome` isn't included for obvious reasons). `pulse`, `pain`, `lesion_1`, `total_protein`, `mucous_membrane`, `temp_of_extremities`, `surgical_lesion`, `abdominal_distention`, `capillary_refill_time`, `peristalsis`, `abdomen`, `abdomo_apperance`, `rectal_exam_fexes`, `abdomo_protein`, and `age` are predicitive of `outcome` in the training set and overall dataset, while not being exceptionally predictive of `outcome` in the testing set. Some variables, such as `mucous_membrane` and `pulse`, have `p < 0.0001` for both the training set and overall dataset, while having `p > 0.25` in the testing set.

Let's view the contigency tables for `mucous_membrane` and `outcome` for all three datasets for more visibility into this behavior.

In [ ]:
train_ct, test_ct, combined_ct = pd.crosstab(horse_train["mucous_membrane"], horse_train["outcome"]), pd.crosstab(horse_test["mucous_membrane"], horse_test["outcome"]), pd.crosstab(horse_df["mucous_membrane"], horse_df["outcome"])
print("Train CT:\n", train_ct)
print("Test CT:\n", test_ct)
print("Combined CT:\n", combined_ct)

The contigency tables show that certain values for `mucous_membrane`, such as `bright_red` and `dark_cyanotic`, have a significant amount of horses that `died` in both the training set and overall dataset but not the testing set. Additionally, the expected values for most of entries in the testing set contigency table are less than five, suggesting that results from the Chi Squared test are unreliable for that subset. With only 75 entries in the dataset, and with three columns and six rows, one can expect a nonnegligible number of entries to have expected values less than five, regardless of the distribution of data (all data being evenly distributed results in an expected value of `75/18 == ~4.2` for all cells).

One solution would be to remove all high-cardinality variables, where for this dataset a high-cardinality column would be one with five or more values in the training set. This threshold will be used because:
1. `75/15 == 5`, so there's the potential for a significant amount of cells to have expected values less than 5 with this many potential values for a categorical variable.
2. Since the training set contains only 75% of the total dataset, there's the potential for the testing set to contain values for variables not encountered in the training set, further increasing the cardinality of the variable. Reasoning behind why the entire dataset shouldn't be used  (and why the previous p-value calculations for the testing set and overall dataset is bad practice) will be provided later.

Let's check how many dimensions this would eliminate.


In [ ]:
for var in horse_train.columns:
    if horse_train[var].dtype not in ["int64", "float64"]:
        print(var, len(horse_train[var].value_counts()))

`pain`, `mucous_membrane`, `abdomen`, and `lesion_1` are high-cardinality variables, and thus are candidates for removal. These are the variables with the 3rd, 7th, 13th, and 4th lowest p-values in the training set. Discarding these columns is discarding valuable information, especially concerning `pain` and `lesion_1`.

Another viable option would be to increase the training/test split from 25% to some higher number, but making all of these changes so that a Chi Squared calculation works for the testing set is nonsensical.

The testing set not being representative of the overall dataset is a nuisance, but the testing set exists to ensure that a model is robust and that it doesn't overfit to any data. If this `DecisionTree` model were trained on the entire dataset, and it was used to predict the `outcome` of novel horses outside of the dataset, the model should be robust enough to achieve satisfactory accuracy even if those novel horses have trends that were not observered in the original dataset. If this model was deployed in practice and it was found to have only 50% accuracy for the first 100 horses it predicts, the argumment could be made that "it's okay, those horses weren't representative of most horses", but such an argument would be weak and meaningless, considering a model that only predicts the data that it expects to be presented is worthless. That's the purpose of a testing set, so there is not much purpose behind the previous p-value calculations of the testing set besides to confirm that there are indeed trends in the training set that aren't in the testing set. But again, since trends like this wouldn't be known in practice, it would be erroneous to use any information about the testing set to inform decisions about the model. The testing set should be viewed as a black box.

Nevertheless, the argument could still be made that this specific seed that was chosen for the train/test split (so that this notebook could be deterministic) produces diminished performance, and that a different train/test split would lead to more impressive metrics. One could also propose that in practice, this model would achieve better results, since this specific train/test split results in unusually incongruent subsets and in practice future data should be expected to be more aligned with the training set - though such a claim would need more evidence than that which is currently available in this notebook. Both of these statements may be true, and the potential validity of such statements is why k-fold Cross-Validation, or similar cross-validation techniques, should be used to evaluate the performance of a model instead of a single random train/test split.

Regardless, let's see how the model would perform given the impossible assumption of clairvoyance and use those variables which are the most predicitive in the testing set to train the `DecisionTree` model. Some previously evaluated models will also be trained again, for convenience of comparison.

In [ ]:
print("Five Dimensions, Height == 5")
build_and_eval_dt(horse_train, horse_test, 5, ["surgery", "age", "surgical_lesion", "lesion_1", "pain"], "outcome", ["lived", "died", "euthanized"])
build_and_eval_dt_imputation(horse_train, horse_test, 5, ["surgery", "age", "surgical_lesion", "lesion_1", "pain"], "outcome", ["lived", "died", "euthanized"])
print("Ten Dimensions, Height == 7")
build_and_eval_dt(horse_train, horse_test, 7, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen"], "outcome", ["lived", "died", "euthanized"])
build_and_eval_dt_imputation(horse_train, horse_test, 7, ["surgery", "age", "surgical_lesion", "lesion_1", "pain", "peristalsis", "pulse", "respiratory_rate", "mucous_membrane", "abdomen"], "outcome", ["lived", "died", "euthanized"])
print("Two Dimensions, Height == 3")
build_and_eval_dt(horse_train, horse_test, 3, ["packed_cell_volume", "peripheral_pulse"], "outcome", ["lived", "died", "euthanized"])
build_and_eval_dt_imputation(horse_train, horse_test, 3, ["packed_cell_volume", "peripheral_pulse"], "outcome", ["lived", "died", "euthanized"])
print("Three Dimensions, Height == 3")
build_and_eval_dt(horse_train, horse_test, 3, ["packed_cell_volume", "peripheral_pulse", "pain"], "outcome", ["lived", "died", "euthanized"])
build_and_eval_dt_imputation(horse_train, horse_test, 3, ["packed_cell_volume", "peripheral_pulse", "pain"], "outcome", ["lived", "died", "euthanized"])
print("Four Dimensions, Height == 4")
build_and_eval_dt(horse_train, horse_test, 4, ["packed_cell_volume", "peripheral_pulse", "pain", "total_protein"], "outcome", ["lived", "died", "euthanized"])
build_and_eval_dt_imputation(horse_train, horse_test, 4, ["packed_cell_volume", "peripheral_pulse", "pain", "total_protein"], "outcome", ["lived", "died", "euthanized"])

After running the prior code, the following results are obtained, with the naive model listed as well for convenience.

```
Naive Model - One Decision Node
Testing Recall: [1.0, 0.0, 0.0]
Testing Precision: [0.6666666666666666, nan, nan]
Testing Accuracy: 0.6666666666666666
Testing Weighted F1: nan

Five Dimensions - Height == 5 (No Imputation)
Testing Recall: [0.62, 0.9285714285714286, 0.0]
Testing Precision: [0.9393939393939394, 0.325, 0.0]
Testing Accuracy: 0.5866666666666667
Testing Weighted F1: nan

Five Dimensions - Height == 5 (With Imputation)
Testing Recall: [0.76, 0.7142857142857143, 0.0]
Testing Precision: [0.8837209302325582, 0.3333333333333333, 0.0]
Testing Accuracy: 0.64
Testing Weighted F1: nan

Ten Dimensions - Height == 7 (No Imputation)
Testing Recall: [0.72, 0.7142857142857143, 0.18181818181818182]
Testing Precision: [0.8780487804878049, 0.3448275862068966, 0.4]
Testing Accuracy: 0.64
Testing Weighted F1: 0.6290391004344493

Ten Dimensions - Height == 7 (With Imputation)
Testing Recall: [0.72, 0.42857142857142855, 0.18181818181818182]
Testing Precision: [0.75, 0.3, 0.2857142857142857]
Testing Accuracy: 0.5866666666666667
Testing Weighted F1: 0.5850624694322174

Two Predictive Dimensions - Height == 3 (No Imputation)
Testing Recall: [0.86, 0.5, 0.09090909090909091]
Testing Precision: [0.7413793103448276, 0.4666666666666667, 0.5]
Testing Accuracy: 0.68
Testing Weighted F1: 0.7164567573762974

Two Predictive Dimensions - Height == 3 (With Imputation)
Testing Recall: [0.92, 0.5, 0.0]
Testing Precision: [0.7301587301587301, 0.5833333333333334, nan]
Testing Accuracy: 0.7066666666666667
Testing Weighted F1: nan

Three Predictive Dimensions - Height == 3 (No Imputation)
Testing Recall: [0.88, 0.5714285714285714, 0.0]
Testing Precision: [0.7857142857142857, 0.4444444444444444, 0.0]
Testing Accuracy: 0.6933333333333334
Testing Weighted F1: nan

Three Predictive Dimensions - Height == 3 (With Imputation)
Testing Recall: [0.92, 0.5714285714285714, 0.0]
Testing Precision: [0.7540983606557377, 0.5714285714285714, nan]
Testing Accuracy: 0.72
Testing Weighted F1: nan

Four Predictive Dimensions - Height == 4 (No Imputation)
Testing Recall: [0.84, 0.5714285714285714, 0.09090909090909091]
Testing Precision: [0.75, 0.4444444444444444, 1.0]
Testing Accuracy: 0.68
Testing Weighted F1: 0.7139203354297695

Four Predictive Dimensions - Height == 4 (With Imputation)
Testing Recall: [0.82, 0.5, 0.09090909090909091]
Testing Precision: [0.7192982456140351, 0.5, 0.25]
Testing Accuracy: 0.6533333333333333
Testing Weighted F1: 0.6828743509865005
```

While the `DecisionTree` with a height of three and the dimensions `packed_cell_volume`, `peripheral_pulse`, and `pain` achieves the highest accuracy of all previous models, it fails to properly classify any `euthanized` horses. There is also no obvious trend in preformance as the number of dimensions increases.

It's possible that because the variables that had the most predictive power for the testing set were chosen, the model was unable to learn certain trends as they may not have been present in the training set. The p-values for these variables are also flawed, due to the low expected values making the Chi Squared Test of Independence unreliable. 

Below, we'll take a look at training the `DecisionTree` model using the top ten most predictive variables ordered by their p-values in the training data subset. The model will be trained using only the most predictive variable, then the two most predictive variables, and so on until ten `DecisionTree` models have been trained, each with a length equal to the dimensionality of the dataset plus one.

In [ ]:
top_ten_predictive = ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities', 'surgical_lesion', 'abdominal_distention']
cols = []
for var in top_ten_predictive:
    cols.append(var)
    print("With variables: ", cols)
    build_and_eval_dt_imputation(horse_train, horse_test, len(cols) + 1, cols, "outcome", ["lived", "died", "euthanized"])

After running the previous code, the following results are obtained:

```
With variables:  ['pulse']
Training Recall: [0.890625, 0.5238095238095238, 0.0], Testing Recall: [0.68, 0.5, 0.0]
Training Precision: [0.7125, 0.515625, nan], Testing Precision: [0.7083333333333334, 0.25925925925925924, nan]
Training Accuracy: 0.65625, Testing Accuracy: 0.5466666666666666
Training Weighted F1: nan, Testing Weighted F1: nan

With variables:  ['pulse', 'peripheral_pulse']
Training Recall: [0.859375, 0.6190476190476191, 0.15151515151515152], Testing Recall: [0.6, 0.5714285714285714, 0.0]
Training Precision: [0.7534246575342466, 0.5492957746478874, 0.7142857142857143], Testing Precision: [0.6976744186046512, 0.25806451612903225, 0.0]
Training Accuracy: 0.6875, Testing Accuracy: 0.5066666666666667
Training Weighted F1: 0.715645694559009, Testing Weighted F1: nan

With variables:  ['pulse', 'peripheral_pulse', 'pain']
Training Recall: [0.828125, 0.8571428571428571, 0.24242424242424243], Testing Recall: [0.7, 0.7857142857142857, 0.0]
Training Precision: [0.8548387096774194, 0.6067415730337079, 0.7272727272727273], Testing Precision: [0.7954545454545454, 0.36666666666666664, 0.0]
Training Accuracy: 0.75, Testing Accuracy: 0.6133333333333333
Training Weighted F1: 0.7658674215300155, Testing Weighted F1: nan

With variables:  ['pulse', 'peripheral_pulse', 'pain', 'lesion_1']
Training Recall: [0.7890625, 0.9523809523809523, 0.2727272727272727], Testing Recall: [0.64, 0.6428571428571429, 0.0]
Training Precision: [0.9439252336448598, 0.5607476635514018, 0.9], Testing Precision: [0.7619047619047619, 0.28125, 0.0]
Training Accuracy: 0.7589285714285714, Testing Accuracy: 0.5466666666666666
Training Weighted F1: 0.7664728855421437, Testing Weighted F1: nan

With variables:  ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume']
Training Recall: [0.8671875, 0.8888888888888888, 0.45454545454545453], Testing Recall: [0.68, 0.7142857142857143, 0.0]
Training Precision: [0.925, 0.6363636363636364, 0.9375], Testing Precision: [0.8292682926829268, 0.3125, 0.0]
Training Accuracy: 0.8125, Testing Accuracy: 0.5866666666666667
Training Weighted F1: 0.8146731981467167, Testing Weighted F1: nan

With variables:  ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein']
Training Recall: [0.984375, 0.8095238095238095, 0.6060606060606061], Testing Recall: [0.76, 0.7857142857142857, 0.2727272727272727]
Training Precision: [0.8811188811188811, 0.864406779661017, 0.9090909090909091], Testing Precision: [0.8636363636363636, 0.4074074074074074, 0.75]
Training Accuracy: 0.8794642857142857, Testing Accuracy: 0.6933333333333334
Training Weighted F1: 0.8852769580096268, Testing Weighted F1: 0.6888303061754022

With variables:  ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane']
Training Recall: [0.984375, 0.9206349206349206, 0.6363636363636364], Testing Recall: [0.76, 0.7142857142857143, 0.2727272727272727]
Training Precision: [0.9197080291970803, 0.8923076923076924, 0.9545454545454546], Testing Precision: [0.8636363636363636, 0.4, 0.5]
Training Accuracy: 0.9151785714285714, Testing Accuracy: 0.68
Training Weighted F1: 0.9195781039420485, Testing Weighted F1: 0.6735017061925698

With variables:  ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities']
Training Recall: [0.96875, 0.9206349206349206, 0.9090909090909091], Testing Recall: [0.72, 0.7142857142857143, 0.36363636363636365]
Training Precision: [0.992, 0.9508196721311475, 0.7894736842105263], Testing Precision: [0.8571428571428571, 0.4166666666666667, 0.4444444444444444]
Training Accuracy: 0.9464285714285714, Testing Accuracy: 0.6666666666666666
Training Weighted F1: 0.9451198055988616, Testing Weighted F1: 0.6546819221967963

With variables:  ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities', 'surgical_lesion']
Training Recall: [0.96875, 0.9682539682539683, 0.7878787878787878], Testing Recall: [0.68, 0.8571428571428571, 0.2727272727272727]
Training Precision: [0.9841269841269841, 0.8472222222222222, 1.0], Testing Precision: [0.8717948717948718, 0.3870967741935484, 0.6]
Training Accuracy: 0.9419642857142857, Testing Accuracy: 0.6533333333333333
Training Weighted F1: 0.9419890310321379, Testing Weighted F1: 0.6427478152309613

With variables:  ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities', 'surgical_lesion', 'abdominal_distention']
Training Recall: [0.984375, 0.9841269841269841, 0.7878787878787878], Testing Recall: [0.74, 0.7857142857142857, 0.2727272727272727]
Training Precision: [0.9921259842519685, 0.8732394366197183, 1.0], Testing Precision: [0.8409090909090909, 0.4230769230769231, 0.6]
Training Accuracy: 0.9553571428571429, Testing Accuracy: 0.68
Training Weighted F1: 0.9559045943193871, Testing Weighted F1: 0.677510638297872
```
There is an obvious correlation between dimensionality and model performance in the training set, with the addition of dimensions almost always leading to an increase in training accuracy and weighted F1, but such a trend only weakly extends to the performance of the model in the testing set. 

It's possible that the specific train/test split that was chosen so this notebook could be determinisitic happens to result in exceptionally poor performance for the `DecisionTree` model. As a final evaluation to rule this out - __but more importantly to obtain more robust metrics that are more representative of the models__ - the same previous `DecisionTree` models will be evaluated using k-fold Cross-Validation. A 25% split will still be used, and the average performance of the models will be obtained across all of the four splits.

In [ ]:
from sklearn.model_selection import KFold


def build_and_return_dt_imputation(df_train, df_test, height, columns, prediction_column, conf_matrix_cols):
    columns = columns.copy()
    columns.append(prediction_column)
    df_train_stripped = df_train.drop(columns=[el for el in df_train.columns.to_list() if el not in columns])
    df_test_stripped = df_test.drop(columns=[el for el in df_test.columns.to_list() if el not in columns])
    for category in columns:
        if df_train[category].dtype in ["int64", "float64"]:
            df_train_stripped[category].fillna(df_train_stripped[category].mean(), inplace=True)
            df_test_stripped[category].fillna(df_train_stripped[category].mean(), inplace=True)
        else:
            df_train_stripped[category].fillna(df_train_stripped[category].mode()[0], inplace=True)
            df_test_stripped[category].fillna(df_train_stripped[category].mode()[0], inplace=True)
    model = DecisionTree(df_train_stripped, prediction_column, height)
    model.train()
    train_conf_matrix = model.predict_df(df_train_stripped, conf_matrix_cols)
    test_conf_matrix = model.predict_df(df_test_stripped, conf_matrix_cols)
    return [get_recalls(train_conf_matrix), get_recalls(test_conf_matrix), get_precisions(train_conf_matrix), get_precisions(test_conf_matrix),
            get_accuracy(train_conf_matrix), get_accuracy(test_conf_matrix), get_weighted_f1(train_conf_matrix), get_weighted_f1(test_conf_matrix)]

# The following code takes a while to execute. The output of the code is shared below.
'''
splits = 4

top_ten_predictive = ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities', 'surgical_lesion', 'abdominal_distention']
kf = KFold(n_splits=splits, shuffle=True, random_state=100)

dt_metrics = {}

for train, test in kf.split(horse_df):
    cols = []
    for var in top_ten_predictive:
        cols.append(var)
        print("With variables: ", cols)
        metrics = build_and_return_dt_imputation(horse_df.iloc[train], horse_df.iloc[test], len(cols) + 1, cols, "outcome", ["lived", "died", "euthanized"])
        dt_metrics.setdefault(len(cols), []).append(metrics)
        
for tree in dt_metrics:
    print("\nFor the Decision Tree with {0} dimension(s):".format(tree))
    metrics = np.sum(dt_metrics[tree], axis=0)
    print("Average Training Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[0]), 4), axis=0) / splits))
    print("Average Testing Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[1]), 4), axis=0) / splits))
    print("Average Training Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[2]), 4), axis=0) / splits))
    print("Average Testing Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[3]), 4), axis=0) / splits))
    print("Average Training Accuracy: {0}".format(metrics[4] / splits))
    print("Average Testing Accuracy: {0}".format(metrics[5] / splits))
    print("Average Training Weighted F1: {0}".format(metrics[6] / splits))
    print("Average Testing Weighted F1: {0}".format(metrics[7] / splits))
'''

```
For the Decision Tree with 1 dimension(s):
Average Training Recalls: [0.9615622  0.23101263 0.02142857]
Average Testing Recalls: [0.89258772 0.13541667 0.02777778]
Average Training Precisions: [0.64552197 0.79178504        nan]
Average Testing Precisions: [0.60497585        nan        nan]
Average Training Accuracy: 0.6376587301587302
Average Testing Accuracy: 0.555045045045045
Average Training Weighted F1: nan
Average Testing Weighted F1: nan

For the Decision Tree with 2 dimension(s):
Average Training Recalls: [0.90822737 0.48235923 0.09659767]
Average Testing Recalls: [0.85931391 0.32787698 0.05555556]
Average Training Precisions: [0.71225816 0.65008118        nan]
Average Testing Precisions: [0.65593326 0.42767857        nan]
Average Training Accuracy: 0.6822817460317461
Average Testing Accuracy: 0.5850450450450451
Average Training Weighted F1: nan
Average Testing Weighted F1: nan

For the Decision Tree with 3 dimension(s):
Average Training Recalls: [0.90603865 0.67348104 0.21398133]
Average Testing Recalls: [0.82713816 0.51686508 0.02083333]
Average Training Precisions: [0.77888802 0.68821966 0.83035714]
Average Testing Precisions: [0.70878172 0.4862069  0.05      ]
Average Training Accuracy: 0.7458382936507937
Average Testing Accuracy: 0.6222522522522522
Average Training Weighted F1: 0.7737768836167283
Average Testing Weighted F1: nan

For the Decision Tree with 4 dimension(s):
Average Training Recalls: [0.92123137 0.56440354 0.3226461 ]
Average Testing Recalls: [0.85419643 0.29563492 0.04166667]
Average Training Precisions: [0.77646239 0.75153749 0.91461353]
Average Testing Precisions: [0.66175214 0.39947917        nan]
Average Training Accuracy: 0.7446974206349206
Average Testing Accuracy: 0.5752702702702703
Average Training Weighted F1: 0.7732241202057514
Average Testing Weighted F1: nan

For the Decision Tree with 5 dimension(s):
Average Training Recalls: [0.90083453 0.8397984  0.41676136]
Average Testing Recalls: [0.77960996 0.44990079 0.09722222]
Average Training Precisions: [0.88337337 0.70275844 0.96966912]
Average Testing Precisions: [0.72574111 0.38255084 0.16071429]
Average Training Accuracy: 0.8160615079365079
Average Testing Accuracy: 0.5786486486486486
Average Training Weighted F1: 0.8241768445171276
Average Testing Weighted F1: nan

For the Decision Tree with 6 dimension(s):
Average Training Recalls: [0.98098386 0.79330276 0.75977408]
Average Testing Recalls: [0.79361842 0.42113095 0.24873737]
Average Training Precisions: [0.88710003 0.90585985 0.96765734]
Average Testing Precisions: [0.68649905 0.51342557 0.42767857]
Average Training Accuracy: 0.8995982142857143
Average Testing Accuracy: 0.6085585585585586
Average Training Weighted F1: 0.9030738109628461
Average Testing Weighted F1: 0.6293162790499458

For the Decision Tree with 7 dimension(s):
Average Training Recalls: [0.95866824 0.92295391 0.84883658]
Average Testing Recalls: [0.75174342 0.53670635 0.24873737]
Average Training Precisions: [0.95348942 0.87563465 0.97222222]
Average Testing Precisions: [0.73108351 0.46630719 0.42376374]
Average Training Accuracy: 0.9330654761904762
Average Testing Accuracy: 0.6187837837837837
Average Training Weighted F1: 0.934019358240149
Average Testing Weighted F1: 0.625015515869016

For the Decision Tree with 8 dimension(s):
Average Training Recalls: [0.96941762 0.94464574 0.93887987]
Average Testing Recalls: [0.7270708  0.52281746 0.27146465]
Average Training Precisions: [0.97806474 0.9417553  0.92709815]
Average Testing Precisions: [0.71751457 0.47991228 0.3452381 ]
Average Training Accuracy: 0.9587152777777778
Average Testing Accuracy: 0.6021171171171171
Average Training Weighted F1: 0.9584012988931355
Average Testing Weighted F1: 0.6063316519472126

For the Decision Tree with 9 dimension(s):
Average Training Recalls: [0.98861607 0.95228752 0.8766572 ]
Average Testing Recalls: [0.73802162 0.52380952 0.35290404]
Average Training Precisions: [0.96961152 0.94791667 0.98648649]
Average Testing Precisions: [0.72351415 0.45959471 0.48603896]
Average Training Accuracy: 0.963219246031746
Average Testing Accuracy: 0.6188738738738739
Average Training Weighted F1: 0.9635515251617208
Average Testing Weighted F1: 0.6259386876934633

For the Decision Tree with 10 dimension(s):
Average Training Recalls: [0.99082902 0.97866128 0.8922822 ]
Average Testing Recalls: [0.76171523 0.50545635 0.37373737]
Average Training Precisions: [0.98035399 0.9475681  1.        ]
Average Testing Precisions: [0.72814105 0.49029304 0.47305195]
Average Training Accuracy: 0.9732589285714286
Average Testing Accuracy: 0.6322522522522522
Average Training Weighted F1: 0.9736749457236334
Average Testing Weighted F1: 0.6387179197242248
```

While the assumption was that the specific train/test split being used for all prior calculations was leading to worse performance metrics than what might otherwise be expected, k-fold Cross-Validation seems to suggest the opposite. Both training and testing accuracy are significantly worse for the `DecisionTree` models after using k-fold Cross-Validation, while the trends noted in the validations prior to this one seem to also be present here. The model achieves best results with a smaller number of dimensions, while still having enough dimensions to accurately model the data. With all ten dimensions the model is able to achieve its best performance, with an average testing accuracy of `0.63` and a weighted F1 of `0.64`. The naive model which always guesses `lived` would result in a testing accuracy of `0.59`, which all models trained on more than five dimensions are able to slightly surpass.

After utilizing k-fold Cross-Validation, all of the `DecisionTree` models still achieve mediocre recall for the `died` and `euthanized` classes. At this point, it's relatively safe to conclude that a simple decision tree is ill-advised for this particular dataset without major enhancements.

However, before we reach this conclusion, we'll build _one_ final model in an attempt to construct something that achieves satisfactory performance. The basic `DecisionTree` class will be enhanced so that it can better handle the imbalanced classes.

Normally, all classes are weighted equally, and so information gain for a decision tree is calculated as `IG(Y, X) == E(Y) - E(Y | X)`. More precisely, this can be written as: 

`IG() == E(ParentNode) - ((E(ChildMemberNode) * (ChildMemberSize / ParentSize))  + (E(ChildNonmemberNode) * (ChildNonmemberSize / ParentSize)))`

When different classes all had the same weight, `ChildMemberSize`, `ChildNonmemberSize`, and `ParentSize` just corresponded to the number of items in each of those nodes. For example, if `ParentNode` had filtered a dataset down to `Xp` items, and the split resulted in `Xm` items filtered down into `ChildMemberNode` and `Xn` items filtered down into `ChildNonmemberNode`, then `ChildMemeberSize == Xm`, `ChildNonmemberSize == Xn`, and `ParentSize == Xp`. With the addition of class weights, these variables would now be the sum of the weighted items, instead of just the sum of all items. For this specific dataset, that would mean `ParentSize == (LivedHorsesInParent * LivedWeight) + (DiedHorsesInParent * DiedWeight) + (EuthanizedHorsesInParent * EuthanizedWeight)`. 

A new class, `DecisionTreeWeighted`, will be added, along with a new `IG()` function, `get_information_gain_weighted()`. Then, a basic model will be trained for both `DecisionTree` and `DecisionTreeWeighted`, and the layout of the two trees will be compared below.

In [ ]:
def get_information_gain_weighted(value_counts_before, value_counts_after_arr, class_weights):
    if any([el.empty for el in value_counts_after_arr]):
        return 0
    row_count = sum([value_counts_before[i] * class_weights.get(value_counts_before.index[i], 1) for i in range(len(value_counts_before))])
    entropy_before = get_entropy(value_counts_before, row_count)
    entropy_after = sum([get_entropy(value_counts, row_count)*(sum([value_counts[i] * class_weights.get(value_counts.index[i], 1) for i in range(len(value_counts))])/row_count) for value_counts in value_counts_after_arr])
    return entropy_before - entropy_after


class DecisionTreeWeighted(DecisionTree):
    def __init__(self, df, prediction_column, height, weights):
        super().__init__(df, prediction_column, height)
        self.weights = weights

    def branch(self, node):
        df = node.df
        input_columns = df.columns.tolist()
        input_columns.remove(self.prediction_column)
        for column in input_columns:
            for unique_value in horse_train[column].unique():
                subsets = [df.loc[df[column] == unique_value], df.loc[df[column] != unique_value]]
                ig = get_information_gain_weighted(df[self.prediction_column].value_counts(), [el[self.prediction_column].value_counts() for el in subsets], self.weights)
                if node.ig < ig or (node.ig == ig and node.ig > 0):
                    node.column = column
                    node.split_value = unique_value
                    node.ig = ig
                    node.conditional = "=="
                    node.member = Node(subsets[0].copy(), node.level + 1)
                    node.non_member = Node(subsets[1].copy(), node.level + 1)
                if horse_train[column].dtype in ["int64", "float64"]:
                    subsets = [df.loc[df[column] >= unique_value], df.loc[df[column] < unique_value]]
                    ig = get_information_gain_weighted(df[self.prediction_column].value_counts(), [el[self.prediction_column].value_counts() for el in subsets], self.weights)
                    if node.ig < ig:
                        node.column = column
                        node.split_value = unique_value
                        node.ig = ig
                        node.conditional = ">="
                        node.member = Node(subsets[0].copy(), node.level + 1)
                        node.non_member = Node(subsets[1].copy(), node.level + 1)
                    subsets = [df.loc[df[column] < unique_value], df.loc[df[column] >= unique_value]]
                    ig = get_information_gain_weighted(df[self.prediction_column].value_counts(), [el[self.prediction_column].value_counts() for el in subsets], self.weights)
                    if node.ig < ig:
                        node.column = column
                        node.split_value = unique_value
                        node.ig = ig
                        node.conditional = "<"
                        node.member = Node(subsets[0].copy(), node.level + 1)
                        node.non_member = Node(subsets[1].copy(), node.level + 1)

            
horse_train_stripped = horse_train.drop(columns=[el for el in horse_train.columns.to_list() if el not in ["surgery", "age", "outcome", "surgical_lesion"]])     
model = DecisionTree(horse_train_stripped, "outcome", 3)
model.train()
print(model.predict({"surgery": "no", "age": "adult", "surgical_lesion": "yes"}))
model_weighted = DecisionTreeWeighted(horse_train_stripped, "outcome", 3, {"lived": 1, "died": 2, "euthanized": 5})
model_weighted.train()
print(model_weighted.predict({"surgery": "no", "age": "adult", "surgical_lesion": "yes"}))
model.display()
model_weighted.display()


In [ ]:
def build_and_return_dt_imputation_weighted(df_train, df_test, height, columns, prediction_column, conf_matrix_cols, weights):
    if prediction_column not in columns:
        columns.append(prediction_column)
    df_train_stripped = df_train.drop(columns=[el for el in df_train.columns.to_list() if el not in columns])
    df_test_stripped = df_test.drop(columns=[el for el in df_test.columns.to_list() if el not in columns])
    for category in columns:
        if df_train[category].dtype in ["int64", "float64"]:
            df_train_stripped[category].fillna(df_train_stripped[category].mean(), inplace=True)
            df_test_stripped[category].fillna(df_train_stripped[category].mean(), inplace=True)
        else:
            df_train_stripped[category].fillna(df_train_stripped[category].mode()[0], inplace=True)
            df_test_stripped[category].fillna(df_train_stripped[category].mode()[0], inplace=True)
    model = DecisionTreeWeighted(df_train_stripped, prediction_column, height, weights)
    model.train()
    train_conf_matrix = model.predict_df(df_train_stripped, conf_matrix_cols)
    test_conf_matrix = model.predict_df(df_test_stripped, conf_matrix_cols)
    return [get_recalls(train_conf_matrix), get_recalls(test_conf_matrix), get_precisions(train_conf_matrix), get_precisions(test_conf_matrix),
            get_accuracy(train_conf_matrix), get_accuracy(test_conf_matrix), get_weighted_f1(train_conf_matrix), get_weighted_f1(test_conf_matrix)]

# The following code takes a while to execute. The output of the code is shared below.
'''
splits = 4

top_ten_predictive = ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities', 'surgical_lesion', 'abdominal_distention']
weights = {"lived": 1, "died": 3, "euthanized": 5}
kf = KFold(n_splits=splits, shuffle=True, random_state=100)

dt_metrics = {}

for train, test in kf.split(horse_df):
    cols = []
    for var in top_ten_predictive:
        cols.append(var)
        print("With variables: ", cols[1:])
        metrics = build_and_return_dt_imputation_weighted(horse_df.iloc[train], horse_df.iloc[test], len(cols) + 1, cols, "outcome", ["lived", "died", "euthanized"], weights)
        dt_metrics.setdefault(len(cols), []).append(metrics)
        
for tree in dt_metrics:
    print("\nFor the Decision Tree with {0} dimension(s):".format(tree-1))
    metrics = np.sum(dt_metrics[tree], axis=0)
    print("Average Training Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[0]), 4), axis=0) / splits))
    print("Average Testing Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[1]), 4), axis=0) / splits))
    print("Average Training Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[2]), 4), axis=0) / splits))
    print("Average Testing Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[3]), 4), axis=0) / splits))
    print("Average Training Accuracy: {0}".format(metrics[4] / splits))
    print("Average Testing Accuracy: {0}".format(metrics[5] / splits))
    print("Average Training Weighted F1: {0}".format(metrics[6] / splits))
    print("Average Testing Weighted F1: {0}".format(metrics[7] / splits))
'''

```
For the Decision Tree with 1 dimension(s):
Average Training Recalls: [0.96513363 0.22035025 0.02924107]
Average Testing Recalls: [0.88737939 0.125      0.02777778]
Average Training Precisions: [0.64590995 0.83531469        nan]
Average Testing Precisions: [0.60364379        nan        nan]
Average Training Accuracy: 0.6387797619047619
Average Testing Accuracy: 0.5483333333333333
Average Training Weighted F1: nan
Average Testing Weighted F1: nan

For the Decision Tree with 2 dimension(s):
Average Training Recalls: [0.89018968 0.52754015 0.15037879]
Average Testing Recalls: [0.83383459 0.36359127 0.02777778]
Average Training Precisions: [0.72335908 0.6507326  0.925     ]
Average Testing Precisions: [0.67145765 0.38455882        nan]
Average Training Accuracy: 0.6900843253968254
Average Testing Accuracy: 0.5816216216216216
Average Training Weighted F1: 0.7305699744432368
Average Testing Weighted F1: nan

For the Decision Tree with 3 dimension(s):
Average Training Recalls: [0.92770559 0.61420992 0.31744453]
Average Testing Recalls: [0.83341635 0.38541667 0.        ]
Average Training Precisions: [0.76543662 0.75921427 0.91742081]
Average Testing Precisions: [0.67680269 0.4518315  0.        ]
Average Training Accuracy: 0.7592311507936509
Average Testing Accuracy: 0.5886036036036035
Average Training Weighted F1: 0.7813881511468939
Average Testing Weighted F1: nan

For the Decision Tree with 4 dimension(s):
Average Training Recalls: [0.97921532 0.58537235 0.41443452]
Average Testing Recalls: [0.87615602 0.24652778 0.11111111]
Average Training Precisions: [0.76676148 0.89460784 0.91098485]
Average Testing Precisions: [0.65862255 0.37580214 0.375     ]
Average Training Accuracy: 0.7960267857142858
Average Testing Accuracy: 0.5952702702702704
Average Training Weighted F1: 0.8163746914574177
Average Testing Weighted F1: nan

For the Decision Tree with 5 dimension(s):
Average Training Recalls: [0.99040179 0.68565721 0.50802895]
Average Testing Recalls: [0.85999373 0.33184524 0.04166667]
Average Training Precisions: [0.80244356 0.95467172 0.97140523]
Average Testing Precisions: [0.67917316 0.45486111 0.09821429]
Average Training Accuracy: 0.8417063492063492
Average Testing Accuracy: 0.5918018018018019
Average Training Weighted F1: 0.8531684095232013
Average Testing Weighted F1: nan

For the Decision Tree with 6 dimension(s):
Average Training Recalls: [0.98644832 0.7589714  0.74833604]
Average Testing Recalls: [0.81204887 0.39037698 0.25568182]
Average Training Precisions: [0.86062598 0.9605364  0.99074074]
Average Testing Precisions: [0.66055128 0.58472222 0.36937229]
Average Training Accuracy: 0.8941121031746032
Average Testing Accuracy: 0.6086036036036035
Average Training Weighted F1: 0.8977577776219388
Average Testing Weighted F1: 0.6350279340483493

For the Decision Tree with 7 dimension(s):
Average Training Recalls: [0.99414062 0.82750786 0.78601867]
Average Testing Recalls: [0.80514411 0.42013889 0.23295455]
Average Training Precisions: [0.89035777 0.98770492 1.        ]
Average Testing Precisions: [0.67412913 0.56121795 0.33333333]
Average Training Accuracy: 0.9220089285714286
Average Testing Accuracy: 0.6086036036036035
Average Training Weighted F1: 0.9244838042220134
Average Testing Weighted F1: 0.6325172246097687

For the Decision Tree with 8 dimension(s):
Average Training Recalls: [0.99821429 0.86433333 0.83175054]
Average Testing Recalls: [0.79273026 0.44444444 0.2260101 ]
Average Training Precisions: [0.91276368 0.99390244 0.99166667]
Average Testing Precisions: [0.66619971 0.57163889 0.37462121]
Average Training Accuracy: 0.9398462301587301
Average Testing Accuracy: 0.6086036036036037
Average Training Weighted F1: 0.9413222207426881
Average Testing Weighted F1: 0.6277206432100837

For the Decision Tree with 9 dimension(s):
Average Training Recalls: [0.99821429 0.88467064 0.88553166]
Average Testing Recalls: [0.79273026 0.44444444 0.2260101 ]
Average Training Precisions: [0.93152383 1.         0.98275862]
Average Testing Precisions: [0.66869518 0.57163889 0.35795455]
Average Training Accuracy: 0.9532192460317461
Average Testing Accuracy: 0.6086036036036037
Average Training Weighted F1: 0.954239132101949
Average Testing Weighted F1: 0.627321530862442

For the Decision Tree with 10 dimension(s):
Average Training Recalls: [0.99620864 0.93519068 0.93016098]
Average Testing Recalls: [0.78326598 0.45833333 0.2260101 ]
Average Training Precisions: [0.96160267 0.99603175 0.99193548]
Average Testing Precisions: [0.68027382 0.56417112 0.32715201]
Average Training Accuracy: 0.9721825396825396
Average Testing Accuracy: 0.6052702702702702
Average Training Weighted F1: 0.9727690634485477
Average Testing Weighted F1: 0.6213812174494402
```

After adding support for class weights to the decision tree model and training the `DecisionTreeWeighted` models with weights of `{"lived": 1, "died": 3, "euthanized": 5}`, the performance of the model marginally regressed for the testing sets. Interestingly, the performance of the model remained similar for the training sets, however. There was also no obvious change in testing recall with the addition of weights, despite the intuition behind the addition of weights.

This may be because there was little reasoning behind the weights that were chosen for the different classes, apart from that classes with fewer representatives should have greater weights. So instead of picking arbitrary weights, weights will be picked based on their inverse frequency. For example, the weight of class `X` will be `Training Set Size / Occurence of X in Training Set`. This will still result in underrepresented classes getting larger weights, but should result in more informed learning than before when arbitrary weights were selected. 

In [ ]:
# The following code takes a while to execute. The output of the code is shared below.
'''
splits = 4

dt_metrics = {}

top_ten_predictive = ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities', 'surgical_lesion', 'abdominal_distention']
kf = KFold(n_splits=splits, shuffle=True, random_state=100)

for train, test in kf.split(horse_df):
    cols = []
    train_subset = horse_df.iloc[train]
    test_subset = horse_df.iloc[test]
    weights = {train_subset["outcome"].value_counts().index[i]: sum(train_subset["outcome"].value_counts())/train_subset["outcome"].value_counts()[i] for i in range(len(train_subset["outcome"].value_counts()))}
    for var in top_ten_predictive:
        cols.append(var)
        print("With variables: ", cols[1:])
        metrics = build_and_return_dt_imputation_weighted(train_subset, test_subset, len(cols) + 1, cols, "outcome", ["lived", "died", "euthanized"], weights)
        dt_metrics.setdefault(len(cols), []).append(metrics)
        
for tree in dt_metrics:
    print("\nFor the Decision Tree with {0} dimension(s):".format(tree-1))
    metrics = np.sum(dt_metrics[tree], axis=0)
    print("Average Training Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[0]), 4), axis=0) / splits))
    print("Average Testing Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[1]), 4), axis=0) / splits))
    print("Average Training Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[2]), 4), axis=0) / splits))
    print("Average Testing Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[3]), 4), axis=0) / splits))
    print("Average Training Accuracy: {0}".format(metrics[4] / splits))
    print("Average Testing Accuracy: {0}".format(metrics[5] / splits))
    print("Average Training Weighted F1: {0}".format(metrics[6] / splits))
    print("Average Testing Weighted F1: {0}".format(metrics[7] / splits))
'''

```
For the Decision Tree with 1 dimension(s):
Average Training Recalls: [0.96513363 0.21638199 0.02924107]
Average Testing Recalls: [0.88737939 0.125      0.02777778]
Average Training Precisions: [0.64478966 0.8334517         nan]
Average Testing Precisions: [0.60364379        nan        nan]
Average Training Accuracy: 0.6376636904761905
Average Testing Accuracy: 0.5483333333333333
Average Training Weighted F1: nan
Average Testing Weighted F1: nan

For the Decision Tree with 2 dimension(s):
Average Training Recalls: [0.8955895  0.51313889 0.1260349 ]
Average Testing Recalls: [0.83978697 0.33382937 0.02777778]
Average Training Precisions: [0.72287399 0.64109919 0.93181818]
Average Testing Precisions: [0.66289208 0.36884454        nan]
Average Training Accuracy: 0.6867361111111111
Average Testing Accuracy: 0.574954954954955
Average Training Weighted F1: 0.7302173415305512
Average Testing Weighted F1: nan

For the Decision Tree with 3 dimension(s):
Average Training Recalls: [0.9332203  0.58742421 0.31630141]
Average Testing Recalls: [0.84532112 0.3735119  0.        ]
Average Training Precisions: [0.75823232 0.7882389  0.90208476]
Average Testing Precisions: [0.67173974 0.47422078 0.        ]
Average Training Accuracy: 0.7558829365079365
Average Testing Accuracy: 0.5919369369369369
Average Training Weighted F1: 0.7801308938190428
Average Testing Weighted F1: nan

For the Decision Tree with 4 dimension(s):
Average Training Recalls: [0.97097356 0.60983669 0.45997024]
Average Testing Recalls: [0.86848058 0.28819444 0.07638889]
Average Training Precisions: [0.77491838 0.90309201 0.97916667]
Average Testing Precisions: [0.68387267 0.37034314 0.19642857]
Average Training Accuracy: 0.8049255952380953
Average Testing Accuracy: 0.591891891891892
Average Training Weighted F1: 0.82152768136283
Average Testing Weighted F1: nan

For the Decision Tree with 5 dimension(s):
Average Training Recalls: [0.99436813 0.69489685 0.51560471]
Average Testing Recalls: [0.86499373 0.38740079 0.04166667]
Average Training Precisions: [0.80638436 0.96063367 0.97140523]
Average Testing Precisions: [0.69679224 0.48373016 0.09821429]
Average Training Accuracy: 0.8472867063492064
Average Testing Accuracy: 0.6084684684684685
Average Training Weighted F1: 0.8581094589023409
Average Testing Weighted F1: nan

For the Decision Tree with 6 dimension(s):
Average Training Recalls: [0.98260216 0.80202528 0.77863907]
Average Testing Recalls: [0.79993578 0.45585317 0.21212121]
Average Training Precisions: [0.88130743 0.95498084 0.99074074]
Average Testing Precisions: [0.66629551 0.62575758 0.30208333]
Average Training Accuracy: 0.9075049603174603
Average Testing Accuracy: 0.611936936936937
Average Training Weighted F1: 0.9100024787298393
Average Testing Weighted F1: 0.6343915386225758

For the Decision Tree with 7 dimension(s):
Average Training Recalls: [0.99414062 0.82750786 0.8163217 ]
Average Testing Recalls: [0.79323935 0.42013889 0.23484848]
Average Training Precisions: [0.89732078 0.98770492 1.        ]
Average Testing Precisions: [0.67194356 0.57447552 0.30769231]
Average Training Accuracy: 0.9264732142857143
Average Testing Accuracy: 0.601936936936937
Average Training Weighted F1: 0.9287990365157612
Average Testing Weighted F1: 0.6234251325354934

For the Decision Tree with 8 dimension(s):
Average Training Recalls: [0.99821429 0.85585876 0.83175054]
Average Testing Recalls: [0.7808255  0.43055556 0.20517677]
Average Training Precisions: [0.90962889 0.99390244 0.99166667]
Average Testing Precisions: [0.65651834 0.57461508 0.31130952]
Average Training Accuracy: 0.9376140873015874
Average Testing Accuracy: 0.5952702702702702
Average Training Weighted F1: 0.9391953165330376
Average Testing Weighted F1: 0.6143825588551789

For the Decision Tree with 9 dimension(s):
Average Training Recalls: [0.99626116 0.89287618 0.88553166]
Average Testing Recalls: [0.7773136  0.44444444 0.2260101 ]
Average Training Precisions: [0.93483181 0.99590164 0.98275862]
Average Testing Precisions: [0.6643512  0.57604895 0.34821429]
Average Training Accuracy: 0.9543353174603175
Average Testing Accuracy: 0.5986036036036037
Average Training Weighted F1: 0.9553034161275
Average Testing Weighted F1: 0.6143929597288649

For the Decision Tree with 10 dimension(s):
Average Training Recalls: [0.99816176 0.93942797 0.93016098]
Average Testing Recalls: [0.78326598 0.45833333 0.2260101 ]
Average Training Precisions: [0.96355511 1.         0.99193548]
Average Testing Precisions: [0.68027382 0.56417112 0.32715201]
Average Training Accuracy: 0.9744146825396824
Average Testing Accuracy: 0.6052702702702702
Average Training Weighted F1: 0.9750009513538302
Average Testing Weighted F1: 0.6213812174494402
```

The `DecisionTreeWeighted` model seems to struggle slightly more with the inverse frequency weights than it did without weights, or with aribtary weights.

Let's try one final combination of weights before concluding this evaluation. To add more weight to underrepresented classes, class weights will be calculated as `(Training Set Size / Occurence of X in Training Set)^2`. This will amplify the differences between classes, encouraging the `DecisionTreeWeighted` model to learn those classes with fewer samples, such as `euthanized`.

In [ ]:
# The following code takes a while to execute. The output of the code is shared below.
'''
splits = 4

dt_metrics = {}

top_ten_predictive = ['pulse', 'peripheral_pulse', 'pain', 'lesion_1', 'packed_cell_volume', 'total_protein', 'mucous_membrane', 'temp_of_extremities', 'surgical_lesion', 'abdominal_distention']
kf = KFold(n_splits=splits, shuffle=True, random_state=100)

for train, test in kf.split(horse_df):
    cols = []
    train_subset = horse_df.iloc[train]
    test_subset = horse_df.iloc[test]
    weights = {train_subset["outcome"].value_counts().index[i]: (sum(train_subset["outcome"].value_counts())/train_subset["outcome"].value_counts()[i])**2 for i in range(len(train_subset["outcome"].value_counts()))}
    for var in top_ten_predictive:
        cols.append(var)
        print("With variables: ", cols[1:])
        metrics = build_and_return_dt_imputation_weighted(train_subset, test_subset, len(cols) + 1, cols, "outcome", ["lived", "died", "euthanized"], weights)
        dt_metrics.setdefault(len(cols), []).append(metrics)
        
for tree in dt_metrics:
    print("\nFor the Decision Tree with {0} dimension(s):".format(tree-1))
    metrics = np.sum(dt_metrics[tree], axis=0)
    print("Average Training Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[0]), 4), axis=0) / splits))
    print("Average Testing Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[1]), 4), axis=0) / splits))
    print("Average Training Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[2]), 4), axis=0) / splits))
    print("Average Testing Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[3]), 4), axis=0) / splits))
    print("Average Training Accuracy: {0}".format(metrics[4] / splits))
    print("Average Testing Accuracy: {0}".format(metrics[5] / splits))
    print("Average Training Weighted F1: {0}".format(metrics[6] / splits))
    print("Average Testing Weighted F1: {0}".format(metrics[7] / splits))
'''

```
For the Decision Tree with 1 dimension(s):
Average Training Recalls: [0.96553082 0.23277912 0.06783009]
Average Testing Recalls: [0.89237939 0.13541667 0.04861111]
Average Training Precisions: [0.65046358 0.82420357        nan]
Average Testing Precisions: [0.60539607        nan        nan]
Average Training Accuracy: 0.6465724206349206
Average Testing Accuracy: 0.5583783783783783
Average Training Weighted F1: nan
Average Testing Weighted F1: nan

For the Decision Tree with 2 dimension(s):
Average Training Recalls: [0.87944484 0.51563539 0.14276245]
Average Testing Recalls: [0.79874687 0.45287698 0.        ]
Average Training Precisions: [0.71040364 0.64638094 0.93303571]
Average Testing Precisions: [0.6758243  0.45546498        nan]
Average Training Accuracy: 0.6789236111111111
Average Testing Accuracy: 0.5783333333333334
Average Training Weighted F1: 0.7204695361773343
Average Testing Weighted F1: nan

For the Decision Tree with 3 dimension(s):
Average Training Recalls: [0.93883499 0.54996913 0.27818588]
Average Testing Recalls: [0.83490445 0.31696429 0.02777778]
Average Training Precisions: [0.73752446 0.77813388 0.96428571]
Average Testing Precisions: [0.66538422 0.3481685         nan]
Average Training Accuracy: 0.7436011904761906
Average Testing Accuracy: 0.565
Average Training Weighted F1: 0.7716566327630492
Average Testing Weighted F1: nan

For the Decision Tree with 4 dimension(s):
Average Training Recalls: [0.97070312 0.50039435 0.37493912]
Average Testing Recalls: [0.86315163 0.24107143 0.02777778]
Average Training Precisions: [0.73929198 0.90500585 0.98863636]
Average Testing Precisions: [0.64172794 0.40833333        nan]
Average Training Accuracy: 0.7648263888888889
Average Testing Accuracy: 0.565045045045045
Average Training Weighted F1: 0.79420212204121
Average Testing Weighted F1: nan

For the Decision Tree with 5 dimension(s):
Average Training Recalls: [0.99448529 0.54549549 0.46825622]
Average Testing Recalls: [0.87404135 0.18303571 0.02083333]
Average Training Precisions: [0.7579964  0.95399306 0.98611111]
Average Testing Precisions: [0.6315882  0.37946429 0.125     ]
Average Training Accuracy: 0.8015823412698413
Average Testing Accuracy: 0.5684234234234234
Average Training Weighted F1: 0.8206678194367489
Average Testing Weighted F1: nan

For the Decision Tree with 6 dimension(s):
Average Training Recalls: [0.99230769 0.71275638 0.70280032]
Average Testing Recalls: [0.82395363 0.39632937 0.23484848]
Average Training Precisions: [0.83476987 0.97560976 1.        ]
Average Testing Precisions: [0.65644641 0.65555556 0.36602564]
Average Training Accuracy: 0.8784871031746032
Average Testing Accuracy: 0.611936936936937
Average Training Weighted F1: 0.8834640095029729
Average Testing Weighted F1: 0.6423377883709119

For the Decision Tree with 7 dimension(s):
Average Training Recalls: [1.         0.73214365 0.74943858]
Average Testing Recalls: [0.83395363 0.39632937 0.21212121]
Average Training Precisions: [0.85133574 1.         1.        ]
Average Testing Precisions: [0.6588346  0.64824561 0.36245421]
Average Training Accuracy: 0.8952232142857143
Average Testing Accuracy: 0.6152702702702703
Average Training Weighted F1: 0.8995237294977234
Average Testing Weighted F1: 0.6479815480095169

For the Decision Tree with 8 dimension(s):
Average Training Recalls: [0.99821429 0.78506914 0.7797822 ]
Average Testing Recalls: [0.82079574 0.40674603 0.23295455]
Average Training Precisions: [0.87296376 0.99390244 1.        ]
Average Testing Precisions: [0.6599895  0.63125427 0.37912088]
Average Training Accuracy: 0.9119444444444444
Average Testing Accuracy: 0.6152702702702703
Average Training Weighted F1: 0.9145503790180449
Average Testing Weighted F1: 0.6424659146732676

For the Decision Tree with 9 dimension(s):
Average Training Recalls: [0.99804688 0.81066149 0.79517045]
Average Testing Recalls: [0.80921679 0.42063492 0.23295455]
Average Training Precisions: [0.88483347 0.99576271 1.        ]
Average Testing Precisions: [0.66032345 0.61634199 0.37912088]
Average Training Accuracy: 0.9208581349206348
Average Testing Accuracy: 0.6118918918918919
Average Training Weighted F1: 0.9229326419885687
Average Testing Weighted F1: 0.6371729586560071

For the Decision Tree with 10 dimension(s):
Average Training Recalls: [1.         0.84175223 0.82571023]
Average Testing Recalls: [0.78243108 0.43452381 0.23295455]
Average Training Precisions: [0.90145753 1.         1.        ]
Average Testing Precisions: [0.6561658  0.57514881 0.35412088]
Average Training Accuracy: 0.9342460317460318
Average Testing Accuracy: 0.5985585585585586
Average Training Weighted F1: 0.9356557473764939
Average Testing Weighted F1: 0.621774762977159
```

These models seem to perform similarly to the previous `DecisionTreeWeighted` models.

Before concluding all of this testing, we'll build _one_ final model.

Recall that the predicitiveness of variables was determined by obtained their p-values using One-way Anova and Chi Squared Test of Independence for variables in the training set. Now that there are `k` different training sets, for the k-fold Cross-Validation, the predictive power of the variables should be recalculated for each training subset. Note that this doesn't make our previous models invalid or flawed, as we were choosing `d` variables that we could've simply chosen at random. However, choosing the dimensions using this method may lead to better performance, so the `DecisionTreeWeighted` model will be rebuilt, with Inverse Frequency to determine class weights and with dimensions selected using p-values to determmine their association with the `outcome` variable.

In [ ]:
def get_most_predictive(df, n):
    ordered_vars = []
    for var in df.columns:
        if df[var].dtype not in ["int64", "float64"]:
            train_p = compute_p_chi2(df, var)
        else:
            train_p = compute_p_anova(df, var)
        bisect.insort(ordered_vars, (train_p, var))
    return ordered_vars[1:n+1]

# The following code takes a while to execute. The output of the code is shared below.
'''
splits = 4

dt_metrics = {}

kf = KFold(n_splits=splits, shuffle=True, random_state=100)

for train, test in kf.split(horse_df):
    cols = []
    train_subset = horse_df.iloc[train]
    test_subset = horse_df.iloc[test]
    top_ten_predictive = get_most_predictive(train_subset, 10)
    weights = {train_subset["outcome"].value_counts().index[i]: (sum(train_subset["outcome"].value_counts())/train_subset["outcome"].value_counts()[i]) for i in range(len(train_subset["outcome"].value_counts()))}
    for _, var in top_ten_predictive:
        cols.append(var)
        metrics = build_and_return_dt_imputation_weighted(train_subset, test_subset, len(cols) + 1, cols, "outcome", ["lived", "died", "euthanized"], weights)
        dt_metrics.setdefault(len(cols), []).append(metrics)
        
for tree in dt_metrics:
    print("\nFor the Decision Tree with {0} dimension(s):".format(tree-1))
    metrics = np.sum(dt_metrics[tree], axis=0)
    print("Average Training Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[0]), 4), axis=0) / splits))
    print("Average Testing Recalls: {0}".format(np.sum(np.split(np.asarray(metrics[1]), 4), axis=0) / splits))
    print("Average Training Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[2]), 4), axis=0) / splits))
    print("Average Testing Precisions: {0}".format(np.sum(np.split(np.asarray(metrics[3]), 4), axis=0) / splits))
    print("Average Training Accuracy: {0}".format(metrics[4] / splits))
    print("Average Testing Accuracy: {0}".format(metrics[5] / splits))
    print("Average Training Weighted F1: {0}".format(metrics[6] / splits))
    print("Average Testing Weighted F1: {0}".format(metrics[7] / splits))
'''

```
For the Decision Tree with 1 dimension(s):
Average Training Recalls: [0.90372445 0.52780697 0.07544643]
Average Testing Recalls: [0.83392544 0.41865079 0.02083333]
Average Training Precisions: [0.71432763 0.59646577        nan]
Average Testing Precisions: [0.6773115  0.45904558        nan]
Average Training Accuracy: 0.6856001984126984
Average Testing Accuracy: 0.5986486486486486
Average Training Weighted F1: nan
Average Testing Weighted F1: nan

For the Decision Tree with 2 dimension(s):
Average Training Recalls: [0.9581905  0.54242586 0.26532062]
Average Testing Recalls: [0.81609492 0.34077381 0.04166667]
Average Training Precisions: [0.74138903 0.77062471 1.        ]
Average Testing Precisions: [0.6410636  0.48559829 0.16666667]
Average Training Accuracy: 0.7502628968253969
Average Testing Accuracy: 0.5718918918918919
Average Training Weighted F1: 0.7809757309767429
Average Testing Weighted F1: nan

For the Decision Tree with 3 dimension(s):
Average Training Recalls: [0.94245591 0.67465996 0.42280168]
Average Testing Recalls: [0.81916197 0.37053571 0.04166667]
Average Training Precisions: [0.79710833 0.81323778 0.94624542]
Average Testing Precisions: [0.63871542 0.51618056 0.20833333]
Average Training Accuracy: 0.7993303571428572
Average Testing Accuracy: 0.5786036036036036
Average Training Weighted F1: 0.812544134628024
Average Testing Weighted F1: nan

For the Decision Tree with 4 dimension(s):
Average Training Recalls: [0.9618349  0.730798   0.61914908]
Average Testing Recalls: [0.81067982 0.38640873 0.13888889]
Average Training Precisions: [0.83138921 0.87189584 1.        ]
Average Testing Precisions: [0.65339772 0.5627107  0.24603175]
Average Training Accuracy: 0.8528521825396825
Average Testing Accuracy: 0.5952702702702702
Average Training Weighted F1: 0.8588710807960152
Average Testing Weighted F1: nan

For the Decision Tree with 5 dimension(s):
Average Training Recalls: [0.99261474 0.74932916 0.64172078]
Average Testing Recalls: [0.7675235  0.40674603 0.1875    ]
Average Training Precisions: [0.84040546 0.95559886 1.        ]
Average Testing Precisions: [0.65661385 0.48515618 0.29464286]
Average Training Accuracy: 0.8785019841269841
Average Testing Accuracy: 0.5753153153153153
Average Training Weighted F1: 0.8847239553299009
Average Testing Weighted F1: nan

For the Decision Tree with 6 dimension(s):
Average Training Recalls: [0.99221755 0.80371705 0.77729978]
Average Testing Recalls: [0.76594455 0.42460317 0.25378788]
Average Training Precisions: [0.88058343 0.97732384 1.        ]
Average Testing Precisions: [0.65377633 0.55559269 0.35714286]
Average Training Accuracy: 0.9130803571428572
Average Testing Accuracy: 0.5919819819819819
Average Training Weighted F1: 0.9162230296974797
Average Testing Weighted F1: 0.6080724521186448

For the Decision Tree with 7 dimension(s):
Average Training Recalls: [0.99414062 0.83530144 0.84556277]
Average Testing Recalls: [0.77984649 0.44593254 0.23484848]
Average Training Precisions: [0.90654945 0.98770492 1.        ]
Average Testing Precisions: [0.6684929  0.55694444 0.34935897]
Average Training Accuracy: 0.9331696428571429
Average Testing Accuracy: 0.5986036036036035
Average Training Weighted F1: 0.935132323191983
Average Testing Weighted F1: 0.619503167955214

For the Decision Tree with 8 dimension(s):
Average Training Recalls: [0.99821429 0.86830159 0.84560335]
Average Testing Recalls: [0.75478383 0.44444444 0.23295455]
Average Training Precisions: [0.91763381 0.98951647 1.        ]
Average Testing Precisions: [0.65789972 0.52474134 0.32733516]
Average Training Accuracy: 0.9431944444444444
Average Testing Accuracy: 0.5852702702702702
Average Training Weighted F1: 0.944698400266125
Average Testing Weighted F1: 0.6021323937408833

For the Decision Tree with 9 dimension(s):
Average Training Recalls: [0.99626116 0.92725418 0.89200487]
Average Testing Recalls: [0.77136122 0.46875    0.23295455]
Average Training Precisions: [0.95054748 0.98994926 0.99074074]
Average Testing Precisions: [0.6780965  0.56370192 0.31964286]
Average Training Accuracy: 0.9643799603174603
Average Testing Accuracy: 0.601981981981982
Average Training Weighted F1: 0.9651669302665186
Average Testing Weighted F1: 0.6166842376753727

For the Decision Tree with 10 dimension(s):
Average Training Recalls: [0.99816176 0.93942797 0.93016098]
Average Testing Recalls: [0.78326598 0.45833333 0.2260101 ]
Average Training Precisions: [0.96355511 1.         0.99193548]
Average Testing Precisions: [0.68285841 0.54901961 0.32715201]
Average Training Accuracy: 0.9744146825396824
Average Testing Accuracy: 0.6052702702702702
Average Training Weighted F1: 0.9750009513538302
Average Testing Weighted F1: 0.621461997317454
```


To compare all of the various decision tree models evaluated with k-fold Cross-Validation, the testing and training accuracies of the models are plotted below.

In [ ]:
import matplotlib.pyplot as plt

train_accuracy_unweighted = [.637, .682, .746, .745, .816, .900, .933, .959, .963, .973]
test_accuracy_unweighted = [.555, .585, .622, .575, .579, .609, .619, .602, .619, .632]

train_accuracy_arbitrary_weights = [.639, .690, .759, .796, .842, .894, .922, .940, .953, .972]
test_accuracy_arbitrary_weights = [.548, .582, .589, .595, .592, .609, .609, .609, .609, .605]

train_accuracy_if_weights = [.639, .687, .756, .805, .847, .908, .926, .938, .954, .974]
test_accuracy_if_weights = [.548, .575, .592, .592, .608, .612, .602, .595, .599, .605]

train_accuracy_if2_weights = [.647, .679, .744, .765, .802, .878, .895, .912, .921, .934]
test_accuracy_if2_weights = [.558, .578, .565, .565, .568, .612, .615, .615, .612, .599]

train_accuracy_significant_if_weights = [.686, .750, .799, .853, .879, .913, .933, .943, .964, .974]
test_accuracy_significant_if_weights = [.599, .572, .579, .595, .575, .591, .599, .585, .602, .605]

fig, ax = plt.subplots(2, figsize=(12, 12))
ax[0].plot([i + 1 for i in range(10)], train_accuracy_unweighted, label="Unweighted", color="red")
ax[0].plot([i + 1 for i in range(10)], train_accuracy_arbitrary_weights, label="Arbitrary Weights", color="blue")
ax[0].plot([i + 1 for i in range(10)], train_accuracy_if_weights, label="Inverse Frequency Weights", color="green")
ax[0].plot([i + 1 for i in range(10)], train_accuracy_if2_weights, label="Inverse Frequency Squared Weights",
           color="black")
ax[0].plot([i + 1 for i in range(10)], train_accuracy_significant_if_weights,
           label="Inverse Frequency Significant Weights", color="magenta")
ax[0].set_ylabel("Training Set Accuracy", fontsize=16)
ax[0].grid()
ax[0].legend(fontsize=12)
ax[1].plot([i + 1 for i in range(10)], test_accuracy_unweighted, label="Unweighted", color="red")
ax[1].plot([i + 1 for i in range(10)], test_accuracy_arbitrary_weights, label="Arbitrary Weights", color="blue")
ax[1].plot([i + 1 for i in range(10)], test_accuracy_if_weights, label="Inverse Frequency Weights", color="green")
ax[1].plot([i + 1 for i in range(10)], test_accuracy_if2_weights, label="Inverse Frequency Squared Weights",
           color="black")
ax[1].plot([i + 1 for i in range(10)], test_accuracy_significant_if_weights,
           label="Inverse Frequency Significant Weights", color="magenta")
ax[1].set_ylabel("Testing Set Accuracy", fontsize=16)
ax[1].set_xlabel("Number of Dimensions", fontsize=16)
ax[1].grid()
ax[1].legend(fontsize=12)
plt.show()

When comparing these five decision tree models there are a few observations worth noting:
1. There is an evident correlation between the number of dimensions a `DecisionTree` is trained on and the accuracy of the model in regards to the training set, regardless of weights.
2. There is an evident - albiet much weaker - correlation between the number of dimensions a `DecisionTree` is trained on and the accuracy of the model in regards to the testing set, regardless of weights.
3. The only model which has a non-significant correlation for accuracy in the testing set is the most recent model, with Inverse Frequency Weights and dimensions chosen by their significance, with a p-value of `p ~= .140`

An explanation for this last observation is that because dimensions are chosen based on their predictiveness in the training set, these dimensions may be causing additional overfitting as the model picks up on noise present in the training set that's not representative of the testing set. On the contrary, if the model was presented with the most predictive dimensions, it would achieve better performance than the other models with fewer dimensions, making the additional performance gain as more dimensions were added less beneficial, and the data seems to reflect this possibility.

Another plausible explanation is that the p-values aren't a reliable measurement of predictiveness. This argument is less sound, but still worth noting. The p-value calculations for the testing set were already shown to be unreliable due to the testing set size, and the p-values for the training set are unreliable for similar reasons. In addition, the p-values from the Chi Squared Test of Independence are being compared with p-values from the One-way Anova during the predictiveness evaluation, which isn't a safe comparison - a p-value from a One-way Anova might be more significant than the same p-value from the Chi Squared Test of Independence, since the p-values don't actually represent "predictiveness". 

At this point, it's reasonable to conclude that a decision tree is poorly suited to this particular dataset. The sparsity, the degree of noise, the limited number of entries and the imbalanced classes of the dataset all make it challenging for a decision tree to properly model the data. Nevertheless, a decision tree is still useful for performing dimensionality reduction, as it can show which variables are more predictive of the output than others based on how often and what layer they were used to partition the dataset. As shown earlier, though, decision trees are still vulnerable to poor perforamnce as a result of overfitting to noisy variables.

A decision tree may still be useful for providing a performant model for this dataset as part of Random Forest or XGBoost, which should be able to overcome many of the shortcomings of a basic decision tree listed above. But a decision tree by itself seems unable to achieve any degree of satisfactory performmance.